In [10]:
import numpy as np
import random
from itertools import permutations
from typing import List, Tuple, Optional, Callable
import time
import pulp
import copy
import yaml
from itertools import combinations
from CBS import load_map, get_random_free_position
from Final_CBS import Environment, CBS

In [11]:
# NOTE: add flag for assigned or not
class CapabilityProfile:
    def __init__(self,
                 robot_id: str,
                 mobility_type: str,
                #  max_speed: float,
                 max_speed: int, # keep speed a constant integer of 1 for CBS purposes for now
                 payload_capacity: Optional[float],
                 reach: Optional[float],
                 battery_life: float,
                 size: Tuple[float, float, float],
                 environmental_resistance: Optional[List[str]],
                 sensors: Optional[List[str]],
                 sensor_range: Optional[float],
                 manipulators: Optional[List[str]],
                 communication_protocols: List[str],
                 processing_power: float,
                 autonomy_level: str,
                 special_functions: Optional[List[str]],
                 safety_features: Optional[List[str]],
                 adaptability: bool,
                 location: Tuple[float, float, float],
                 #preferred_tasks: List[str],
                 current_task: Optional["TaskDescription"],
                 remaining_distance: float,
                 time_on_task: float,
                 tasks_attempted: int,
                 tasks_successful: int,
                 current_path: list, # the path the robot is currently following, if any
                 assigned: bool): # Indicates if the robot is currently assigned to a task
        self.robot_id = robot_id
        self.mobility_type = mobility_type
        self.max_speed = max_speed
        self.payload_capacity = payload_capacity
        self.reach = reach
        self.battery_life = battery_life
        self.size = size  # (length, width, height)
        self.environmental_resistance = environmental_resistance
        self.sensors = sensors
        self.sensor_range = sensor_range
        self.manipulators = manipulators
        self.communication_protocols = communication_protocols
        self.processing_power = processing_power
        self.autonomy_level = autonomy_level
        self.special_functions = special_functions
        self.safety_features = safety_features
        self.adaptability = adaptability
        self.location = location  # (x, y, z) coordinates
        #self.preferred_tasks = preferred_tasks
        self.current_task = current_task  # The task the robot is currently assigned to
        self.remaining_distance = remaining_distance  # Distance left to the current task location
        self.time_on_task = time_on_task  # Time spent on the current task
        self.tasks_attempted = tasks_attempted
        self.tasks_successful = tasks_successful
        self.current_path = current_path
        self.assigned = assigned # says if the robot is assigned to a task to replace the index lists below

    def __repr__(self):
        return (f"CapabilityProfile(robot_id={self.robot_id}, mobility_type={self.mobility_type}, "
                f"max_speed={self.max_speed}, payload_capacity={self.payload_capacity}, "
                f"reach={self.reach}, battery_life={self.battery_life}, size={self.size}, "
                f"environmental_resistance={self.environmental_resistance}, sensors={self.sensors}, "
                f"sensor_range={self.sensor_range}, manipulators={self.manipulators}, communication_protocols={self.communication_protocols}, "
                f"processing_power={self.processing_power}, autonomy_level={self.autonomy_level}, "
                f"special_functions={self.special_functions}, safety_features={self.safety_features}, "
                f"adaptability={self.adaptability}, location={self.location}, remaining_distance={self.remaining_distance}, time_on_task={self.time_on_task}, tasks_attempted={self.tasks_attempted}, tasks_successful={self.tasks_successful})")

class TaskDescription:
    def __init__(self,
                 task_id: str,
                 task_type: str,
                 objective: str,
                 priority_level: str,
                 reward: float,
                 difficulty: float,
                 location: Tuple[float, float, float],
                 navigation_constraints: Optional[List[str]],
                 required_capabilities: List[str],
                 time_window: Optional[Tuple[str, str]],
                 environmental_conditions: Optional[List[str]],
                 dependencies: Optional[List[str]],
                 tools_needed: Optional[List[str]],
                 communication_requirements: Optional[List[str]],
                 safety_protocols: Optional[List[str]],
                 performance_metrics: List[str],
                 success_criteria: str,
                 assigned_robot: Optional["CapabilityProfile"],
                 time_to_complete: float,
                 assigned: bool):
        self.task_id = task_id
        if (task_id in {"maintenance", "cleaning"}): #figure out tasks that won't have progress reset after interruption
            self.reset_progress = False 
        else:
            self.reset_progress = True
        self.task_type = task_type
        self.objective = objective
        self.priority_level = priority_level
        self.reward = reward
        self.difficulty = difficulty
        self.location = location  # (x, y, z) coordinates
        self.navigation_constraints = navigation_constraints
        self.required_capabilities = required_capabilities
        self.time_window = time_window  # (start_time, end_time) or None
        self.environmental_conditions = environmental_conditions
        self.dependencies = dependencies
        self.tools_needed = tools_needed
        self.communication_requirements = communication_requirements
        self.safety_protocols = safety_protocols
        self.performance_metrics = performance_metrics
        self.success_criteria = success_criteria
        self.assigned_robot = assigned_robot  # The robot currently assigned to this task
        self.time_to_complete = time_to_complete  # Total time required to complete the task
        self.time_left = time_to_complete # Counter used during the step method, resets back to time_to_complete based on reset_progress
        self.assigned = assigned

    def __repr__(self):
        return (f"TaskDescription(task_id={self.task_id}, task_type={self.task_type}, "
                f"objective={self.objective}, priority_level={self.priority_level}, "
                f"location={self.location}, reward={self.reward}, difficulty={self.difficulty}, navigation_constraints={self.navigation_constraints}, "
                f"required_capabilities={self.required_capabilities}, time_window={self.time_window}, "
                f"environmental_conditions={self.environmental_conditions}, "
                f"dependencies={self.dependencies}, tools_needed={self.tools_needed}, "
                f"communication_requirements={self.communication_requirements}, "
                f"safety_protocols={self.safety_protocols}, performance_metrics={self.performance_metrics}, "
                f"success_criteria={self.success_criteria}, time_to_complete={self.time_to_complete}), time_left={self.time_left}) ")

In [3]:
def generate_random_robot_profile(robot_id: str, grid: List[List[int]], occupied_locations: set) -> CapabilityProfile:
    num_capabilities = random.randint(1,5)

    mobility_types_list = ["wheeled", "tracked", "legged", "aerial", "hovering", "climbing"]
    environmental_resistances_list = ["weatherproof", "waterproof", "dustproof", "heat-resistant", "cold-resistant", "shock-resistant"]
    sensors_list = ["camera", "microphone", "LiDAR", "GPS", "ultrasonic", "temperature sensor", "infrared", "proximity sensor", "magnetometer"]
    manipulators_list = ["gripper", "drill", "welding tool"]
    communication_protocols = ["Wi-Fi", "Bluetooth", "4G", "5G", "Radio"]
    special_functions_list = ["object recognition", "speech output", "facial recognition", "object tracking", "gesture recognition"]
    safety_features_list = ["emergency stop", "collision avoidance", "overheat protection", "fall detection", "obstacle detection", "speed reduction in crowded areas"]
    #task_preferences = ["delivery", "inspection", "cleaning", "monitoring", "maintenance", "assembly", "surveying", "data collection", "assistance"]
    #find out how to better incorporate task preferences or if its needed at all because it is currently generated randomly, which makes no sense
    
    capabilities = {
        "payload_capacity": random.uniform(20.0, 50.0),
        "reach": random.uniform(3.0, 10.0),
        "environmental_resistance": random.sample(environmental_resistances_list, k=random.randint(0, len(environmental_resistances_list))),
        "sensors": random.sample(sensors_list, k=random.randint(1, len(sensors_list))),
        "manipulators": random.sample(manipulators_list, k=random.randint(0, len(manipulators_list))),
        "special_functions": random.sample(special_functions_list, k=random.randint(0, len(special_functions_list))),
        "safety_features": random.sample(safety_features_list, k=random.randint(0, len(safety_features_list)))
    }

    selected_capabilites = random.sample(list(capabilities.keys()), num_capabilities)
    for key in capabilities:
        if key not in selected_capabilites:
            if key in ["payload_capacity", "reach"]: #zero makes more sense than none for numerical variables
                capabilities[key] = 0
            else:
                capabilities[key] = None

    if capabilities["sensors"] != None:
        sensor_range = random.uniform(20.0, 50.0)
    else:
        sensor_range = None
    
    return CapabilityProfile(
        robot_id=robot_id,
        mobility_type=random.choice(mobility_types_list),
        # max_speed=random.uniform(0.5, 15.0),
        max_speed=1, # constant max speed for now, CBS purposes, macro time steps would increase complexity of time dimension slowing down CBS significantly
        payload_capacity=capabilities["payload_capacity"],
        reach=capabilities["reach"],
        battery_life=random.uniform(50.0, 100.0),
        size=(random.uniform(1.0, 5.0), random.uniform(1.0, 5.0), random.uniform(1.0, 5.0)),
        environmental_resistance=capabilities["environmental_resistance"],
        sensors=capabilities["sensors"],
        sensor_range=sensor_range, 
        manipulators=capabilities["manipulators"],
        communication_protocols=random.sample(communication_protocols, k=random.randint(1, len(communication_protocols))),
        processing_power=random.uniform(5.0, 15.0),
        autonomy_level=random.choice(["teleoperated", "semi-autonomous", "fully autonomous"]),
        special_functions=capabilities["special_functions"],
        safety_features=capabilities["safety_features"],
        adaptability=bool(random.getrandbits(1)),
        # location=(random.uniform(0.0, 50.0), random.uniform(0.0, 50.0), 0.0),
        location=get_random_free_position(grid, occupied_locations), # chooses location from the map were using
        current_task = None,
        remaining_distance = 0.0,
        time_on_task = 0,
        tasks_attempted = 0,
        tasks_successful = 0,
        current_path = [], # holds the path from CBS solution
        assigned=False
    )

def get_unique_task_location(tasks, grid, occupied_locations, max_attempts=100):
    """
    Finds a unique location for a new task that does not overlap with existing tasks or occupied robot locations.
    Occupied locations only include robot locations, not task locations so we check both with this function.
    Args:
        tasks: List of existing TaskDescription objects to check against.
        grid: The grid representing the environment, used to find free positions.
        occupied_locations: A set of tuples representing locations occupied by robots.
        max_attempts: Maximum number of attempts to find a unique location before giving up.

    Returns:
        location: A unique (x, y, z) location tuple for the new task.
    """
    attempts = 0
    task_locations = {task.location for task in tasks}
    all_occupied_locations = occupied_locations | task_locations  # Combine occupied locations with existing task locations
    location = get_random_free_position(grid, all_occupied_locations)
    # Ensure the location is unique and not occupied by any robot or existing task
    while any(np.allclose(location, occ_loc) for occ_loc in all_occupied_locations):
        attempts += 1
        if attempts >= max_attempts:
            raise Exception(f"Could not find a unique task location after {max_attempts} attempts")
        location = get_random_free_position(grid, all_occupied_locations)
    return location

def generate_random_task_description(task_id: str, grid: List[List[int]], occupied_locations: set, tasks) -> TaskDescription:  # occupied_locations has all robot locations, tasks will give us the task locations so that no 2 tasks share a location
    # NOTE: two tasks in the same location can cause CBS deadlock
    task_types = ["delivery", "inspection", "cleaning", "monitoring", "maintenance", "assembly", "surveying", "data collection", "assistance"]
    priorities = ["low", "medium", "high", "urgent"]
    navigation_constraints = ["elevator", "stairs", "shelves", "no loud noises allowed", "narrow spaces", "low ceilings", "uneven floors", "low visibility", "slippery", "crowded", "loose debris", "no-fly zone", "windy", "dense obstructions", "smooth surfaces"]
    environmental_conditions = ["weatherproof", "waterproof", "dustproof", "heat-resistant", "cold-resistant", "shock-resistant"]
    performance_metrics = ["time taken", "accuracy", "energy consumption", "safety compliance", "completion rate"]
    requirements = [
        "payload capacity >= 5.0", 
        "payload capacity >= 1.0", 
        "payload capacity >= 10.0", 
        "payload capacity >= 20.0", 
        "reach >= 1.5", 
        "reach >= 3.0"
    ]
    tools = ["gripper", "camera", "microphone", "temperature sensor", "drill", "welding tool"]
    communication_types = ["Wi-Fi", "Bluetooth", "4G", "5G", "Radio"]
    safety_types = ["emergency stop", "collision avoidance", "overheat protection", "fall detection", "obstacle detection", "speed reduction in crowded areas"]
    duration_time = random.uniform(10.0, 20.0)
    
    task_domains = {
        "delivery": [tools[0], ],
        "inspection": [tools[1], random.choice(requirements[4:])],
        "cleaning": [tools[0], requirements[1]],
        "monitoring": [tools[1], random.choice(requirements[0:3])],
        "maintenance": [tools[5], random.choice(requirements[4:])],
        "assembly": [tools[4], random.choice(requirements[4:])],
        "surveying": [tools[1], random.choice(requirements[0:3])],
        "data collection": [tools[3], random.choice(requirements[0:3])],
        "assistance": [tools[2], random.choice(requirements[0:3])]
    }

    task = random.choice(task_types)
    required = task_domains[task]

    num_capabilities = random.randint(1,3)
    capabilities = {
        "navigation_constraints": random.sample(navigation_constraints, k=random.randint(0, len(navigation_constraints))),
        "environmental_conditions": random.sample(environmental_conditions, k=random.randint(0, len(environmental_conditions))),
        "safety_protocols": random.sample(safety_types, k=random.randint(0, len(safety_types)))
    }

    selected_capabilites = random.sample(list(capabilities.keys()), num_capabilities)
    for key in capabilities:
        if key not in selected_capabilites:
            capabilities[key] = None

    return TaskDescription(
        task_id=task_id,
        task_type=task,
        objective=f"Perform {random.choice(task_types)} task",
        priority_level=random.choice(priorities),
        reward=random.uniform(1, 10),
        difficulty=random.uniform(1, 10),
        # location=(random.uniform(0.0, 50.0), random.uniform(0.0, 50.0), 0.0),
        location = get_unique_task_location(tasks, grid, occupied_locations),
        navigation_constraints=capabilities["navigation_constraints"],
        required_capabilities=required[1],
        time_window=(f"{random.randint(8, 17)}:00", f"{random.randint(18, 23)}:00"),
        environmental_conditions=capabilities["environmental_conditions"],
        dependencies=None,
        tools_needed=required[0],
        communication_requirements=random.sample(communication_types, k=random.randint(1, len(communication_types))),
        safety_protocols=capabilities["safety_protocols"],
        performance_metrics=random.sample(performance_metrics, k=random.randint(1, len(performance_metrics))),
        success_criteria="Task completed within time window",
        assigned_robot = None,
        time_to_complete = duration_time,
        assigned=False
    )

In [4]:
def generate_random_robot_profile_strict(robot_id: str, grid: List[List[int]], occupied_locations: set) -> CapabilityProfile:
    #mobility_types_list = ["wheeled", "tracked", "legged", "aerial", "hovering", "climbing"]
    #environmental_resistances_list = ["weatherproof", "waterproof", "dustproof", "heat-resistant", "cold-resistant", "shock-resistant"]
    #sensors_list = ["camera", "microphone", "LiDAR", "GPS", "ultrasonic", "temperature sensor", "infrared", "proximity sensor", "magnetometer"]
    #manipulators_list = ["gripper", "drill", "welding tool",]
    #communication_protocols = ["Wi-Fi", "Bluetooth", "4G", "5G", "Radio"]
    #special_functions_list = ["object recognition", "speech output", "facial recognition", "object tracking", "gesture recognition"]
    #safety_features_list = ["emergency stop", "collision avoidance", "overheat protection", "fall detection", "obstacle detection", "speed reduction in crowded areas"]
    # task_types = ["delivery", "inspection", "cleaning", "monitoring", "maintenance", "assembly", "surveying", "data collection", "assistance"]

    delivery_profile = {
        "mobility_type": "aerial",
        "environmental_resistances": ["weatherproof", "heat-resistant", "cold-resistant"],
        "sensors": ["LiDAR", "GPS", "proximity sensor", "camera"],
        "manipulators": ["gripper"],
        "communication_protocols": ["Wi-Fi", "4G"],
        "special_functions": ["object recognition", "object tracking", "facial recognition"],
        "safety_features": ["obstacle detection", "emergency stop"],
        "sensor_range": 25.0,
        "processing_power": 5.0,
        "autonomy_level": "fully autonomous",
        "payload_capacity": 20.0,
        "reach": 0.0,
        "battery_life": 80.0,
        "size": (2.0, 2.0, 3.0),
        "adaptability": False
    }

    assembly_profile = {
        "mobility_type": "hovering",
        "environmental_resistances": ["dustproof", "heat-resistant", "shock-resistant"],
        "sensors": ["camera", "proximity sensor", "infrared"],
        "manipulators": ["gripper", "dispenser", "drill", "welding tool"],
        "communication_protocols": ["Wi-Fi", "Radio"],
        "special_functions": ["object recognition", "object tracking", "gesture recognition", "precise alignment"],
        "safety_features": ["collision avoidance", "overheat protection", "emergency stop"],
        "sensor_range": 10.0,
        "processing_power": 7.0,
        "autonomy_level": "semi-autonomous",
        "payload_capacity": 20.0,
        "reach": 5.0,
        "battery_life": 75.0,
        "size": (3.0, 2.0, 3.0),
        "adaptability": True
    }

    bulldozer_profile = {
        "mobility_type": "tracked",
        "environmental_resistances": ["dustproof", "shock-resistant", "weatherproof"],
        "sensors": ["LiDAR", "camera", "proximity sensor", "infrared"],
        "manipulators": ["hydraulic bucket"],
        "communication_protocols": [ "Wi-Fi", "Radio"],
        "special_functions": ["terrain leveling"],
        "safety_features": ["collision avoidance", "emergency stop", "obstacle detection"],
        "sensor_range": 15.0,
        "processing_power": 10,
        "autonomy_level": "semi-autonomous",
        "payload_capacity": 35.0,
        "reach": 1.5,
        "battery_life": 80.0,
        "size": (3.5, 2.5, 2.5),
        "adaptability": True
    }

    excavator_profile = {
        "mobility_type": "tracked",
        "environmental_resistances": ["dustproof", "shock-resistant", "weatherproof"],
        "sensors": ["LiDAR", "camera", "proximity sensor", "magnetometer"],
        "manipulators": ["hydraulic bucket", "cable hoist"],
        "communication_protocols": ["Wi-Fi", "Radio"],
        "special_functions": ["terrain leveling", "object recognition"],
        "safety_features": ["collision avoidance", "overload protection", "emergency stop"],
        "sensor_range": 15.0,
        "processing_power": 8.0,
        "autonomy_level": "semi-autonomous",
        "payload_capacity": 20.0,
        "reach": 8.0,
        "battery_life": 75.0,
        "size": (4.2, 2.5, 3.0),
        "adaptability": True
    }

    bricklayer_profile = {
        "mobility_type": "tracked",
        "environmental_resistances": ["dustproof"],
        "sensors": ["LiDAR", "camera", "proximity sensor"],
        "manipulators": ["dispenser", "gripper"],
        "communication_protocols": ["Wi-Fi", "4G"],
        "special_functions": ["object recognition", "object tracking", "precise alignment"],
        "safety_features": ["collision avoidance", "emergency stop", "overload protection"],
        "sensor_range": 10.0,
        "processing_power": 6.0,
        "autonomy_level": "autonomous",
        "payload_capacity": 15.0,
        "reach": 4.0,
        "battery_life": 80.0,
        "size": (2.0, 1.2, 1.5),
        "adaptability": False
    }

    concrete_profile = {
        "mobility_type": "wheeled",
        "environmental_resistances": ["dustproof", "waterproof", "shock-resistant"],
        "sensors": ["infrared", "camera", "proximity sensor"],
        "manipulators": ["mixer drum", "dispenser"],
        "communication_protocols": ["Wi-Fi", "Radio"],
        "special_functions": ["precise alignment", "concrete mixing"],
        "safety_features": ["obstacle detection", "emergency stop", "collision avoidance"],
        "sensor_range": 10.0,
        "processing_power": 5.5,
        "autonomy_level": "semi-autonomous",
        "payload_capacity": 20.0,  
        "reach": 5.5, 
        "battery_life": 80.0,
        "size": (4.0, 2.2, 2.5),
        "adaptability": False
    }   

    crane_profile = {
        "mobility_type": "tracked",  # stationary crane or one that can move?
        "environmental_resistances": ["weatherproof", "wind-resistant", "dustproof"],
        "sensors": ["camera", "GPS", "proximity sensor", "ultrasonic"],
        "manipulators": ["gripper", "cable hoist"],
        "communication_protocols": ["5G", "Radio"],
        "special_functions": ["precise alignment", "object tracking", "object detection"],
        "safety_features": ["overload protection", "balance control", "collision avoidance", "emergency stop"],
        "sensor_range": 30.0,
        "processing_power": 6.5,
        "autonomy_level": "semi-autonomous",
        "payload_capacity": 35.0,
        "reach": 30.0,
        "battery_life": 70.0,
        "size": (3.5, 3.5, 10.0),
        "adaptability": False
    }

    scaffolding_profile = {
        "mobility_type": "climbing", 
        "environmental_resistances": ["dustproof", "shock-resistant", "wind-resistant"],
        "sensors": ["LiDAR", "camera", "ultrasonic", "proximity sensor"],
        "manipulators": ["gripper", "drill", "welding tool", "cable hoist"],
        "communication_protocols": ["Wi-Fi", "Bluetooth"],
        "special_functions": ["precise alignment", "balance control"],
        "safety_features": ["fall detection", "collision avoidance", "emergency stop"],
        "sensor_range": 12.0,
        "processing_power": 7.0,
        "autonomy_level": "autonomous",
        "payload_capacity": 18.0,  
        "reach": 10.0,  
        "battery_life": 65.0,
        "size": (1.5, 1.0, 2.0),
        "adaptability": True
    }

    robot_profiles = [delivery_profile, assembly_profile, bulldozer_profile, excavator_profile, bricklayer_profile, concrete_profile, crane_profile, scaffolding_profile]
    selected_profile = random.choice(robot_profiles)

    return CapabilityProfile(
        robot_id=robot_id,
        mobility_type=selected_profile["mobility_type"],
        # max_speed=random.uniform(0.5, 15.0),
        max_speed=1, # constant max speed for now, CBS purposes, macro time steps would increase complexity of time dimension slowing down CBS significantly
        payload_capacity=selected_profile["payload_capacity"],
        reach=selected_profile["reach"],
        battery_life=selected_profile["battery_life"],
        size=selected_profile["size"],
        environmental_resistance=selected_profile["environmental_resistances"],
        sensors=selected_profile["sensors"],
        sensor_range=selected_profile["sensor_range"], 
        manipulators=selected_profile["manipulators"],
        communication_protocols=selected_profile["communication_protocols"],
        processing_power=selected_profile["processing_power"],
        autonomy_level=selected_profile["autonomy_level"],
        special_functions=selected_profile["special_functions"],
        safety_features=selected_profile["safety_features"],
        adaptability=selected_profile["adaptability"],
        # location=(random.uniform(0.0, 50.0), random.uniform(0.0, 50.0), 0.0),
        location=get_random_free_position(grid, occupied_locations), # chooses location from the map were using
        current_task = None,
        remaining_distance = 0.0,
        time_on_task = 0,
        tasks_attempted = 0,
        tasks_successful = 0,
        current_path = [], # holds the path from CBS solution
        assigned=False
    )

def generate_random_task_description_strict(task_id: str, grid: List[List[int]], occupied_locations: set, tasks) -> TaskDescription:  # occupied_locations has all robot locations, tasks will give us the task locations so that no 2 tasks share a location
    #task_types = ["delivery", "assembly", "excavate", "level", "item elevation", "lay walls", "scaffold", "remove scaffold"]
    #priorities = ["low", "medium", "high", "urgent"]
    #navigation_constraints = ["elevator", "stairs", "shelves", "no loud noises allowed", "narrow spaces", "low ceilings", "uneven floors", "low visibility", "slippery", "crowded", "loose debris", "no-fly zone", "windy", "dense obstructions", "smooth surfaces"]
    #environmental_conditions = ["weatherproof", "waterproof", "dustproof", "heat-resistant", "cold-resistant", "shock-resistant"]
    #performance_metrics = ["time taken", "accuracy", "energy consumption", "safety compliance", "completion rate"]
    requirements = [
        "payload capacity >= 5.0", 
        "payload capacity >= 1.0", 
        "payload capacity >= 10.0", 
        "payload capacity >= 20.0", 
        "reach >= 1.5", 
        "reach >= 3.0"
    ]
    #tools = ["gripper", "camera", "microphone", "temperature sensor", "drill", "welding tool"]
    #communication_types = ["Wi-Fi", "Bluetooth", "4G", "5G", "Radio"]
    #safety_types = ["emergency stop", "collision avoidance", "overheat protection", "fall detection", "obstacle detection", "speed reduction in crowded areas"]
    #duration_time = random.uniform(10.0, 20.0)

    delivery_profile = {
        "task_type": "delivery",
        "priority_level": random.choice(["low", "medium"]),
        "reward": random.randint(2, 5),
        "difficulty": random.randint(2, 5),
        "navigation_constraints": random.sample(["stairs", "no-fly zone", "narrow spaces", "crowded"], random.randint(0,4)),
        "required_capabilities": random.sample(requirements[0:3], 1),
        "environmental_conditions": random.sample(["weatherproof", "dustproof"], 1),
        "tools_needed": ["camera", "proximity sensor", "GPS", "gripper"],
        "communication_requirements": ["Wi-Fi", "4G"],
        "safety_protocols": ["obstacle detection", "emergency stop"],
        "duration": random.randint(5, 10),
        "performance_metric": "time taken"
    }

    assembly_profile = {
        "task_type": "assembly",
        "priority_level": random.choice(["medium", "high"]),
        "reward": random.randint(5, 7),
        "difficulty": random.randint(5, 7),
        "navigation_constraints": random.sample(["crowded", "low ceilings"], random.randint(0, 2)),
        "required_capabilities": [random.choice(requirements[2:5])],
        "environmental_conditions": random.sample(["dustproof", "shock-resistant", "heat-resistant"], random.randint(0, 3)),
        "tools_needed": ["camera", "infrared", "dispenser", "gripper"] + random.sample(["drill", "welding tool"], 1),
        "communication_requirements": ["Wi-Fi", "Radio"],
        "safety_protocols": ["collision avoidance", "emergency stop"],
        "duration": random.randint(8, 15),
        "performance_metric": "accuracy"
    }

    excavate_profile = {
        "task_type": "excavate",
        "priority_level": random.choice(["high", "urgent"]),
        "reward": random.randint(6, 9),
        "difficulty": random.randint(6, 8),
        "navigation_constraints": ["uneven floors", "loose debris"],
        "required_capabilities": [f"payload >= {random.randint(15, 20)}", f"reach >= {random.randint(2,7)}"],
        "environmental_conditions": ["dustproof", "shock-resistant", "weatherproof"],
        "tools_needed": ["LiDAR", "camera", "proximity sensor", "hydraulic bucket"],
        "communication_requirements": ["Radio"],
        "safety_protocols": ["overload protection", "obstacle detection"],
        "duration": random.randint(10, 18),
        "performance_metric": "safety compliance"
    }

    level_profile = {
        "task_type": "level",
        "priority_level": random.choice(["medium", "high"]),
        "reward": random.randint(4, 7),
        "difficulty": random.randint(4, 6),
        "navigation_constraints": ["uneven floors", "dense obstructions", "loose debris"],
        "required_capabilities": [f"payload >= {random.randint(15, 25)}",],
        "environmental_conditions": ["dustproof", "shock-resistant", "weatherproof"],
        "tools_needed": ["LiDAR", "camera", "proximity sensor", "hydraulic bucket"],
        "communication_requirements": ["Radio", "Wi-Fi"],
        "safety_protocols": ["balance control", "collision avoidance", "emergency stop"],
        "duration": random.randint(10, 18),
        "performance_metric": "completion rate"
    }

    elevation_profile = {
        "task_type": "item elevation",
        "priority_level": random.choice(["low", "medium"]),
        "reward": random.randint(4, 8),
        "difficulty": random.randint(4, 8),
        "navigation_constraints": ["windy", "no-fly zone"],
        "required_capabilities": [f"reach >= {random.randint(15, 25)}", "payload >= 15"],
        "environmental_conditions": ["weatherproof", "wind-resistant"],
        "tools_needed": ["camera", "GPS", "proximity sensor", "ultrasonic", "cable hoist", "gripper"],
        "communication_requirements": ["Radio", "Wi-Fi"],
        "safety_protocols": ["overload protection", "emergency stop"],
        "duration": random.randint(3, 8),
        "performance_metric": "safety compliance"
    }

    walls_profile = {
        "task_type": "lay walls",
        "priority_level": "medium",
        "reward": random.randint(6, 12),
        "difficulty": random.randint(6, 12),
        "navigation_constraints": ["crowded", "narrow spaces"],
        "required_capabilities": ["reach >= 4", "payload >= 10"],
        "environmental_conditions": ["dustproof", "weatherproof"],
        "tools_needed": ["LiDAR", "camera", "proximity sensor", "gripper", "dispenser"],
        "communication_requirements": ["Wi-Fi"],
        "safety_protocols": ["collision avoidance", "overheat protection"],
        "duration": random.randint(8, 20),
        "performance_metric": "accuracy"
    }
    
    scaffold_profile = {
        "task_type": "scaffold",
        "priority_level": "medium",
        "reward": random.randint(5, 8),
        "difficulty": random.randint(5, 8),
        "navigation_constraints": ["narrow spaces", "elevator", "low ceilings"],
        "required_capabilities": ["payload >= 10", "reach >= 3"],
        "environmental_conditions": ["weatherproof", "wind-resistant"],
        "tools_needed": ["gripper", "drill"],
        "communication_requirements": ["LiDAR", "camera", "ultrasonic", "proximity sensor", "Radio", "Wi-Fi"],
        "safety_protocols": ["overload protection", "balance control", "emergency stop"],
        "duration": random.randint(10, 15),
        "performance_metric": "safety compliance"
    }

    remove_scaffold_profile = {
        "task_type": "remove scaffold",
        "priority_level": "medium",
        "reward": random.randint(5, 8),
        "difficulty": random.randint(5, 8),
        "navigation_constraints": ["narrow spaces", "stairs", "loose debris"],
        "required_capabilities": ["payload >= 10", "reach >= 3"],
        "environmental_conditions": ["weatherproof", "dustproof", "wind-resistant"],
        "tools_needed": ["gripper"],
        "communication_requirements": ["LiDAR", "camera", "ultrasonic", "proximity sensor", "Radio", "Wi-Fi"],
        "safety_protocols": ["collision avoidance", "fall detection"],
        "duration": random.randint(10, 15),
        "performance_metric": "safety compliance"
    }

    task_profiles = [delivery_profile, assembly_profile, excavate_profile, level_profile, elevation_profile, walls_profile, scaffold_profile, remove_scaffold_profile]
    selected_profile = random.choice(task_profiles)

    return TaskDescription(
        task_id=task_id,
        task_type=selected_profile["task_type"],
        objective=f"Perform {selected_profile["task_type"]} task",
        priority_level=selected_profile["priority_level"],
        reward=selected_profile["reward"],
        difficulty=selected_profile["difficulty"],
        # location=(random.uniform(0.0, 50.0), random.uniform(0.0, 50.0), 0.0),
        location = get_unique_task_location(tasks, grid, occupied_locations),
        navigation_constraints=selected_profile["navigation_constraints"],
        required_capabilities=selected_profile["required_capabilities"],
        time_window=(f"{random.randint(8, 17)}:00", f"{random.randint(18, 23)}:00"),
        environmental_conditions=selected_profile["environmental_conditions"],
        dependencies=None,
        tools_needed=selected_profile["tools_needed"],
        communication_requirements=selected_profile["communication_requirements"],
        safety_protocols=selected_profile["safety_protocols"],
        performance_metrics=selected_profile["performance_metric"],
        success_criteria="Task completed within time window",
        assigned_robot=None,
        time_to_complete=selected_profile["duration"],
        assigned=False
    )

In [88]:
def evaluate_suitability_new(robot: CapabilityProfile, task: TaskDescription) -> float:
    score = 0.0

    weights = {
        "payload": 3.0,
        "tools": 4.0,
        "communication": 0.5,
        "safety": 1.0,
        "environmental": 1.0,
        "reach": 2.0,
        "sensor_range": 1.0,
        "proximity": 1.0,
        "autonomy_match": 0.5,
        "battery_duration": 2.0,
        "special_functions": 2.0,
        "processing_power": 1.0,
        "adaptability": 0.5,
    }

    if any(req for req in task.required_capabilities if "payload capacity" in req and robot.payload_capacity < float(req.split(">= ")[-1])):
        score += -1.5 * weights["payload"]
        #return 0.0  
    else:
        score += weights["payload"]

    if any(req for req in task.required_capabilities if "reach" in req and robot.reach < float(req.split(">= ")[-1])):
        score += 0.0 
    else:
        score += weights["reach"]  

    if robot.sensors:
        matched_tools = sum(tool in robot.sensors for tool in task.tools_needed)
        tool_score = matched_tools / len(task.tools_needed)
        score += weights["tools"] * tool_score
    if robot.manipulators:
        matched_tools = sum(tool in robot.manipulators for tool in task.tools_needed)
        if matched_tools == 0:
            score += -1.5 * weights["tools"]
            #return 0.0
        else:
            tool_score = matched_tools / len(task.tools_needed)
            score += weights["tools"] * tool_score

    if task.communication_requirements:
        matched_communication = sum(protocol in robot.communication_protocols for protocol in task.communication_requirements)
        communication_score = matched_communication/len(task.communication_requirements)
        score += weights["communication"] * communication_score

    if robot.safety_features and task.safety_protocols:
        matched_safety = sum(safety in robot.safety_features for safety in task.safety_protocols)
        safety_score = matched_safety/len(task.safety_protocols)
        if task.performance_metrics == "safety compliance":
            if safety_score < 0.75:
                score += -1.5 * weights["safety"]
                #return 0.0
        score += weights["safety"] * safety_score

    if robot.environmental_resistance and task.environmental_conditions:
        matched_environmental = sum(condition in robot.environmental_resistance for condition in task.environmental_conditions)
        environmental_score = matched_environmental/len(task.environmental_conditions)
        score += weights["environmental"] * environmental_score

    if task.navigation_constraints:
        navigation_match = navigation_suitability(robot.mobility_type, robot.size, task.navigation_constraints)
        if navigation_match == 0:
            score += -6.0
            #return 0.0
        else:
            score += navigation_match
    
    distance_to_task = len(robot.current_path) - 1
    if robot.sensor_range:
        if robot.sensor_range >= distance_to_task:
            score += weights["sensor_range"]
        elif robot.sensor_range >= distance_to_task/2:
            score += weights["sensor_range"] * 0.5


    if distance_to_task < 20.0:
        score += weights["proximity"]
    elif distance_to_task < 50.0:
        score += 0.5 * weights["proximity"]

    if task.priority_level in ["high", "urgent"] and robot.autonomy_level in ["fully autonomous", "teleoperated"]:
        score += weights["autonomy_match"]
    elif task.priority_level in ["medium", "low"] and robot.autonomy_level in ["semi-autonomous", "fully autonomous"]:
        score += 0.5 * weights["autonomy_match"]

    if ((distance_to_task / robot.max_speed)+task.time_to_complete) > robot.battery_life:
        return 0.0
    if robot.battery_life >= 2*((distance_to_task / robot.max_speed)+task.time_to_complete):
        score += weights["battery_duration"]
    else:
        score += 0.5 * weights["battery_duration"] 

    #task_types = ["delivery", "assembly", "excavate", "level", "item elevation", "lay walls", "scaffold", "remove scaffold"]
    task_function_mapping = {
        "delivery": ["object recognition", "speech output", "facial recognition"],
        "assembly": ["object recognition", "object tracking", "precise alignment"],
        "excavate": [ "terrain leveling", "object recognition", "precise alignment"],
        "level": ["terrain leveling", "object recognition"],
        "item elevation": ["precise alignment", "object tracking", "balance control"],
        "lay walls": ["object recognition", "precise alignment"],
        "scaffold": ["precise alignment", "balance control"],
        "remove scaffold": ["object recognition", "object tracking", "precise alignment"],
    }

    required_functions = task_function_mapping[task.task_type]

    if robot.special_functions:
        matched_special_functions = sum(special in robot.special_functions for special in required_functions)
        special_functions_score = matched_special_functions / len(required_functions)
        score += weights["special_functions"] * special_functions_score

    if task.difficulty > 7 and robot.processing_power >= 5.0:  # Difficulty > 7 indicates a complex task
        score += weights["processing_power"]
    elif task.difficulty > 4 and robot.processing_power >= 3.0:
        score += weights["processing_power"]
    elif task.difficulty > 2 and robot.processing_power >= 1.5:
        score += 0.5 * weights["processing_power"]

    if robot.adaptability:
        score += weights["adaptability"]

    priority_multiplier = {"low": 0.5, "medium": 1.0, "high": 1.5, "urgent": 2.0}[task.priority_level]
    reward_to_difficulty_ratio = task.reward / task.difficulty
    score += priority_multiplier * reward_to_difficulty_ratio

    if score < 0.0:
        return 0.0
    return score


In [114]:
map_file = r"test_small_open.map"
grid = load_map(map_file) # 2D list of 0/1 representing the map
dims = (len(grid), len(grid[0])) # dimensions of the map grid
obstacles = create_obstacle_list(grid) # list of obstacle coordinates
    # Create the map dictionary to pass to the CBS planner
map_dict = {
        'dimension': dims,
        'obstacles': obstacles
}

robots = [generate_random_robot_profile_strict(f"R{i+1}", grid, set()) for i in range(i)]
                            # changed how tasks are made here so there are no overlapping tasks
tasks = [generate_random_task_description_strict(f"T{i+1}", grid, set(), []) for i in range(j)]
suitability_matrix = calculate_suitability_matrix(robots, tasks, sm)

print(suitability_matrix)

[[10.58333333 10.79166667 18.98571429 11.08333333 10.79166667]
 [10.58333333 10.79166667 18.98571429 11.08333333 10.79166667]
 [11.75        7.125      12.78571429  7.41666667  7.125     ]
 [10.58333333 10.79166667 18.98571429 11.08333333 10.79166667]
 [11.75        7.125      12.78571429  7.41666667  7.125     ]]


In [115]:
import psutil

def main_simulation(output: tuple[list[tuple[int, int]],list[int],list[int]], robots: List[CapabilityProfile], tasks: List[TaskDescription], num_candidates: int, voting_method: str, grid: List[List[int]], map_dict: dict, suitability_method: str, suitability_matrix: np.ndarray, max_time_steps: int, add_tasks: bool, add_robots: bool, remove_robots: bool, tasks_to_add: int = 1, robots_to_add: int = 1, robots_to_remove: int = 1):
    print(f"SUITABILITY METHOD: {suitability_method}")
    
    initial_positions = set()
    robot_max_id = len(robots)+1
    task_max_id = len(tasks)+1
    total_reward = 0.0
    total_success = 0.0
    total_tasks = len(tasks)
    total_reassignment_time = 0.0
    total_reassignment_score = 0.0
    total_reassignments = 0

    for r in robots:
        # add the robot's initial position to the occupied positions set
        initial_positions.add(r.location)

    occupied_positions = set(initial_positions) # use the occcupied positions as the current positions for CBS, this is just as a occupation check, not a start position

    assigned_pairs = output[0]  
    for robot_idx, task_idx in assigned_pairs:
        r = robots[robot_idx]
        t = tasks[task_idx]

        r.current_task = t
        r.assigned = True
        r.tasks_attempted = 1

        t.assigned_robot = r
        t.assigned = True

    assigned_robots = {r.robot_id: r.current_task.task_id for r in
                       robots if r.assigned and r.current_task is not None}
    unassigned_tasks = [t.task_id for t in tasks if not t.assigned]
    unassigned_robots = [r.robot_id for r in robots if not r.assigned]

    start_positions = {
        r.robot_id: r.location
        for r in robots
        if r.assigned and r.current_task is not None
    }
    goal_positions = {
        r.robot_id: r.current_task.location
        for r in robots
        if r.assigned and r.current_task is not None
    }

    print(f"ROBOTS: {[rob.robot_id for rob in robots]}")
    print(f"TASKS: {[tas.task_id for tas in tasks]}")
    # print(f"ASSIGNED PAIRS: {assigned_pairs}")
    print(f"ASSIGNED ROBOTS: {assigned_robots}")

    agents = build_cbs_agents(robots, start_positions, goal_positions)

    print(f"AGENTS LIST: {agents}")

    # Create the input data dictionary for CBS, this will be passed to the CBS planner
    input_data = {
        'map' : {
            'dimension': map_dict['dimension'],
            'obstacles': map_dict['obstacles']
        },
        'agents': agents,
    }

    env = Environment(
        dimension=map_dict['dimension'],
        agents=input_data['agents'],
        obstacles=map_dict['obstacles'],
    )
    planner = CBS(env)
    solution, nodes_expanded, conflicts = planner.search()

    print(f"SOLUTION: {solution}")

    if solution is None:
        print("CBS could not find a conflict free path assignment for all agents")
        # could possibly fall back on the simple method here if we get a lot of issues
    else:

        # Iterate through the agents and their schedules
        for agent, schedule in solution.items():
            print(f"Schedule for {agent}:")

            # Iterate through the schedule (list of dictionaries)
            for point in schedule:
                # Access each entry in the schedule
                # print(f"  t: {point['t']}, x: {point['x']}, y: {point['y']}")
                # Assign the paths to the robots
                rid = [robot.robot_id for robot in robots].index(agent)
                robots[rid].current_path.append((point['x'], point['y']))
            print(f"Robot {agent} path: {robots[rid].current_path}")

    # Keep track of previous state to not run CBS when there are no changes
    prev_state = {
        "robot_ids": set(robot.robot_id for robot in robots),
        "assigned": {robot.robot_id for robot in robots if robot.assigned},
        "unassigned": set(unassigned_robots)
    }
 
    # End the simulation if nothing changes for 3 timesteps (hopefully the tasks are done and the robots arent moving)
    time_steps_unchanged = 0

    for time_step in range(max_time_steps):
        # print(f"\n--- Time Step {time_step + 1} ---")
        # print(f"OCCUPIED POSITIONS: {occupied_positions}")
        print(f"START POSITIONS: {start_positions}")
        print(f"GOAL POSITONS: {goal_positions}")
        # print(f"UNASSIGNED ROBOTS: {unassigned_robots}")
        # print(f"UNASSIGNED TASKS: {unassigned_tasks}")
        # # print(f"ASSIGNED ROBOTS: {[rob.assigned for rob in robots]}")
        # print(f"ALL ROBOTS: {len(robots)}")
        # print(f"ALL TASKS: {len(tasks)}")
        # print(f"LIST OF ALL ROBOTS: {[rob.robot_id for rob in robots]}")
        # print(f"LIST OF ALL TASKS: {[tas.task_id for tas in tasks]}")

        # before each time step, refresh the unassigned robots and tasks lists
        unassigned_robots = [r.robot_id for r in robots if not r.assigned]
        unassigned_tasks = [t.task_id for t in tasks if not t.assigned]

        # Simulate time step
        unassigned_count, total_reward, total_success = simulate_time_step(robots, tasks, unassigned_robots, unassigned_tasks, suitability_method, occupied_positions, start_positions, goal_positions, 1.0, total_reward, total_success)
        # Periodically add new tasks and robots
        if add_tasks and time_step + 1 <= 2 and random.random() < 0.5: # add tasks only in the first 2 time steps, and randomly
            print(f"ADDING NEW TASKS AT TIME STEP {time_step + 1}")
            task_max_id, total_tasks = add_new_tasks_strict(tasks, unassigned_tasks, task_max_id, random.randint(0, tasks_to_add), total_tasks, grid, occupied_positions)
        if add_robots and time_step + 1 <= 4 and random.random() < 0.5: # add robots only in the first 4 time steps, and randomly
            print(f"ADDING NEW ROBOTS AT TIME STEP {time_step + 1}")
            robot_max_id = add_new_robots_strict(robots, unassigned_robots, robot_max_id, random.randint(0, robots_to_add), grid, occupied_positions)

        # Periodically remove robots
        if remove_robots and time_step + 1 <= 4 and random.random() < 0.5: # remove robots only in the first 4 time steps, and randomly
            if len(assigned_robots) > 1: # Otherwise will break CBS, we need at least one agent for things to run smoothly
                print(f"REMOVING RANDOM ROBOTS AT TIME STEP {time_step + 1}")
                remove_random_robots(robots, tasks, unassigned_robots, unassigned_tasks, random.randint(0, robots_to_remove), occupied_positions, start_positions, goal_positions)
 

        # Reassign unassigned robots to unassigned tasks
        # TODO: change this to make sure newly assigned robot is more suitable for the task
        # so we will run it when there are unassigned robots AND a state change, otherwise theres no reason to keep checking
        if unassigned_robots and unassigned_tasks:
            total_reassignments += 1
            _, unassigned_robots, unassigned_tasks, reassign_score, reassign_length = reassign_robots_to_tasks(
                robots, tasks, num_candidates, voting_method, suitability_method,
                unassigned_robots, unassigned_tasks, start_positions, goal_positions)
            total_reassignment_time += reassign_length
            total_reassignment_score += reassign_score
            # regenerate the master assigned robots dictionary
            assigned_robots = {
                r.robot_id: r.current_task.task_id
                for r in robots
                if r.assigned and r.current_task is not None
            }

            print(f"ROBOTS REASSIGNED")
            # print(f"NEW ASSIGNMENTS FROM REASSIGNMENT: {new_assignments}")
            print(f"ASSIGNED ROBOTS: {assigned_robots}")
            print(f"ALL ROBOTS: {[robot.robot_id for robot in robots]}")
 
 
        # Update start and goal positions before cbs
        for robot in robots:
            if robot.assigned and robot.current_task:
                start_positions[robot.robot_id] = robot.location
                goal_positions[robot.robot_id] = robot.current_task.location
            else:
                start_positions.pop(robot.robot_id, None)
                goal_positions.pop(robot.robot_id, None)

        current_state = {
            "robot_ids": set(robot.robot_id for robot in robots),
            "assigned": {robot.robot_id for robot in robots if robot.assigned},
            "unassigned": set(unassigned_robots)
        }
 
        if current_state != prev_state and [rob.assigned for rob in robots].count(True) >= 1:
            print("State change, re-run CBS...")
            print(f"AMOUNT OF ASSIGNED ROBOTS: {len([rob.current_task for rob in robots])}")
            print(f"ASSIGNED ROBOTS: {[rob.assigned for rob in robots].count(True)}")
            # NOTE: use occupied positions as current positions for running CBS, update goal positions whenever needed
            print(f"OCCUPIED POSITIONS: {occupied_positions}")
            print(f"START POSITIONS: {start_positions}")
            print(f"GOAL POSITONS: {goal_positions}")
            print(f"UNASSIGNED ROBOTS: {unassigned_robots}")
            print(f"UNASSIGNED TASKS: {unassigned_tasks}")
            print(f"ALL ROBOTS: {len(robots)}")
            print(f"ALL TASKS: {len(tasks)}")
            print(f"LIST OF ALL ROBOTS: {[rob.robot_id for rob in robots]}")
            print(f"LIST OF ALL TASKS: {[tas.task_id for tas in tasks]}")
            agents = build_cbs_agents(robots, start_positions, goal_positions)

            print(f"AGENTS LIST: {agents}")
 
            # Validation block
            start_locations = [agent['start'] for agent in agents]
            if len(start_locations) != len(set(start_locations)):
                print("ERROR: Duplicate start locations found in agent list. Aborting CBS.")
                # You can decide how to handle this. For now, we can skip the planning step for this cycle.
                # To prevent a crash, you could set solution to None and handle it.
                solution = None
            else:
                # instantiate environment and planner
                env = Environment(
                    dimension=map_dict['dimension'],
                    agents=agents,
                    obstacles=map_dict['obstacles']
                )
                planner = CBS(env)
                solution, nodes_expanded, conflicts = planner.search()
                print(f"CBS COMPLETE. New solution: {solution}")

            if solution:
                # update each robot's path based on the new CBS solution
                for robot_id, schedule in solution.items():
                    ridx = [robot.robot_id for robot in robots].index(robot_id)
                    for point in schedule:
                        # print(f"  t: {point['t']}, x: {point['x']}, y: {point['y']}")
                        # Assign the paths to the robots
                        robots[ridx].current_path.append((point['x'], point['y']))
                    # ridx = [r.robot_id for r in robots].index(robot_id)
                    # robots[ridx].current_path = schedule
                    robots[ridx].remaining_distance = len(schedule) - 1
                prev_state = current_state.copy()

            else:
                print("CBS could not find a conflict free path assignment for all agents")
                # could possibly fall back on the simple method here if we get a lot of issues
                # but for now, we will just skip CBS and continue with the simulation
                print("Skipping CBS...")
                time_steps_unchanged += 1
                if time_steps_unchanged >= 3:
                    print("No state change for 3 time steps, ending simulation.")
                    break
        else:
            print("No state change, skip CBS...")
            print(f"ALL ROBOTS: {len(robots)}")
            print(f"ALL TASKS: {len(tasks)}")
            print(f"LIST OF ALL ROBOTS: {[rob.robot_id for rob in robots]}")
            print(f"LIST OF ALL TASKS: {[tas.task_id for tas in tasks]}")
            # if time_steps_unchanged < 3:
            #     time_steps_unchanged += 1
            #     print("No state change, skip CBS...")
            #     print(f"ALL ROBOTS: {len(robots)}")
            #     print(f"ALL TASKS: {len(tasks)}")
            #     print(f"LIST OF ALL ROBOTS: {[rob.robot_id for rob in robots]}")
            #     print(f"LIST OF ALL TASKS: {[tas.task_id for tas in tasks]}")
            # else:
            #     break

    overall_success_rate = total_success / total_tasks
    print(f"Voting: Total reward: {total_reward}, Overall success rate: {overall_success_rate:.2%}, Tasks completed: {total_success}, Reassignment Time: {total_reassignment_time}, Reassignment Score: {total_reassignment_score}, total reassignments: {total_reassignments}")
#     for robot in robots:
#         print(f"Robot {robot.robot_id} attempted {robot.tasks_attempted} tasks and successfully completed {robot.tasks_successful} of them.")

def benchmark_simulation(output: tuple[list[tuple[int, int]],list[int],list[int]], robots: List[CapabilityProfile], tasks: List[TaskDescription], num_candidates: int, voting_method: str, grid: List[List[int]], map_dict: dict, suitability_method: str, suitability_matrix: np.ndarray, max_time_steps: int, add_tasks: bool, add_robots: bool, remove_robots: bool, tasks_to_add: int = 1, robots_to_add: int = 1, robots_to_remove: int = 1):
    start_time = time.time()
    main_simulation(output, robots, tasks, num_candidates, voting_method, grid, map_dict, suitability_method, suitability_matrix, max_time_steps, add_tasks, add_robots, remove_robots, tasks_to_add, robots_to_add, robots_to_remove)
    end_time = time.time()
    execution_time = end_time - start_time

    cpu_usage = psutil.cpu_percent()
    memory_usage = psutil.virtual_memory().used

    print(f"Simulation completed in {execution_time:.2f} seconds.")
    print(f"CPU Usage: {cpu_usage}%")
    print(f"Memory Usage: {memory_usage / (1024 * 1024)} MB")

if __name__ == "__main__":
#     voting_methods = ["rank_assignments_borda", "rank_assignments_approval", "rank_assignments_majority_judgment", "rank_assignments_cumulative_voting", "rank_assignments_condorcet_method", "rank_assignments_range"]
    voting_methods = ["rank_assignments_range"]
#     suitability_methods = ["evaluate_suitability_loose", "evaluate_suitability_strict", "evaluate_suitability_distance", "evaluate_suitability_priority"]
    # suitability_methods = ["evaluate_suitability_loose","evaluate_suitability_distance"]
    suitability_methods = ["evaluate_suitability_new"]
    # suitability_methods = ["evaluate_suitability_distance"]
    max_time_steps = 100
    add_tasks = True
    add_robots = False
    remove_robots = False

    map_file = r"test_small_open.map"
    grid = load_map(map_file) # 2D list of 0/1 representing the map
    dims = (len(grid), len(grid[0])) # dimensions of the map grid
    obstacles = create_obstacle_list(grid) # list of obstacle coordinates
    # Create the map dictionary to pass to the CBS planner
    map_dict = {
        'dimension': dims,
        'obstacles': obstacles
    }

    # print(f"MAP DICTIONARY: {map_dict}")
    for i in [5]:
        for j in [5]:
            for nc in [5]:
                for vm in voting_methods:
                    for sm in suitability_methods:
                        for k in range(0,10):
                            # get initial positions list to reuse later
                            # generate_random_robot_profile now takes in the map grid and occupied positions and passes them to get_random_free_position function
                            robots = [generate_random_robot_profile_strict(f"R{i+1}", grid, set()) for i in range(i)]
                            # changed how tasks are made here so there are no overlapping tasks
                            tasks = [generate_random_task_description_strict(f"T{i+1}", grid, set(), []) for i in range(j)]

                            suitability_matrix = calculate_suitability_matrix(robots, tasks, sm)

                            output, score, length = assign_tasks_with_voting(robots, tasks, suitability_matrix, nc, vm)
                            cbba_output, cbba_score, cbba_length = assign_tasks_with_method(cbba_task_allocation,suitability_matrix)
                            ssia_output, ssia_score, ssia_length = assign_tasks_with_method(ssia_task_allocation,suitability_matrix)
                            ilp_output, ilp_score, ilp_length = assign_tasks_with_method(ilp_task_allocation,suitability_matrix)
                            outputs = [output, cbba_output, ssia_output, ilp_output]
                                # param_combinations = []
                                # param_combinations.append((i, j, nc, vm, sm, max_time_steps, add_tasks, add_robots, remove_robots, 10, 10, 10))
                                # with multiprocessing.Pool() as pool:
                                # pool.starmap(main_simulation, param_combinations)
                                # main_simulation(i, j, nc, vm, sm, max_time_steps, add_tasks, add_robots, remove_robots,10,10,10)
                            for o in outputs:
                                benchmark_simulation(o, copy.deepcopy(robots), copy.deepcopy(tasks), nc, vm, grid, map_dict, sm, suitability_matrix, max_time_steps, add_tasks, add_robots, remove_robots,10,10,10)




Score of the Best Assignment: 65.82
Time taken for the voting process: 17.10 microseconds
Total Suitability Score: 69.65
Time taken for allocation: 96.40 microseconds
Suitability Matrix Issue: [[14.38333333 10.26666667 11.33333333 10.91666667 11.75      ]
 [13.38333333 10.76666667 11.16666667 10.75       12.08333333]
 [ 5.55        3.1         3.5         3.08333333  9.58333333]
 [14.55        3.76666667 20.         19.58333333 21.41666667]
 [14.55        3.76666667 20.         19.58333333 21.41666667]]
Number of Robots: 5, Number of Tasks: 5
Robot 0 bids 14.383333333333335 for Task 0
Robot 3 bids 14.55 for Task 0
Robot 0 bids 10.266666666666666 for Task 1
Robot 1 bids 10.766666666666666 for Task 1
Robot 0 bids 11.333333333333332 for Task 2
Robot 4 bids 20.000000000000004 for Task 2
Robot 0 bids 10.916666666666666 for Task 3
Robot 2 bids 9.583333333333332 for Task 4
Total Suitability Score: 65.82
Time taken for allocation: 492.80 microseconds
Total Suitability Score: 74.32
Time taken f

In [6]:
def navigation_suitability(robot_mobility_type: str, robot_size: Tuple[float, float, float], task_constraints: List[str]) -> float:
    """
    Evaluates the suitability of a robot for navigating a task environment based on mobility type, size, and navigation constraints.
    
    Parameters:
        robot_mobility_type: The mobility type of the robot (e.g., "wheeled", "tracked", "legged", "aerial", "hovering", "climbing").
        robot_size: A tuple representing the robot's dimensions (length, width, height).
        task_constraints: A list of navigation constraints for the task environment.
    
    Returns:
        A float score representing the suitability for navigation. Returns 0 if there is a critical mismatch that prevents navigation.
    """

    # Initialize the score
    score = 0.0

    # Define size thresholds for narrow spaces, low ceilings, etc.
    narrow_space_threshold = 1.0  # Width limit for narrow spaces
    low_ceiling_threshold = 1.5   # Height limit for low ceilings

    # Handle each task constraint based on mobility type
    for constraint in task_constraints:
        # Constraint: Elevator access
        if constraint == "elevator":
            if robot_mobility_type in ["wheeled", "tracked", "legged"]:
                score += 1.0

        # Constraint: Stairs
        elif constraint == "stairs":
            if robot_mobility_type in ["legged", "aerial", "climbing"]:
                score += 1.0

        # Constraint: Shelves
        elif constraint == "shelves":
            if robot_size[2] < low_ceiling_threshold or robot_mobility_type in ["aerial", "climbing", "hovering"]:
                score += 1.0  # Only smaller robots can access shelves effectively

        # Constraint: No loud noises allowed
        elif constraint == "no loud noises allowed":
            if robot_mobility_type in ["legged", "hovering"]:
                score += 1.0  # Quieter mobility types

        # Constraint: Narrow spaces
        elif constraint == "narrow spaces":
            if robot_size[1] <= narrow_space_threshold:
                score += 1.0
            else:
                return 0.0  # Larger robots cannot pass through narrow spaces

        # Constraint: Low ceilings
        elif constraint == "low ceilings":
            if robot_size[2] <= low_ceiling_threshold:
                score += 1.0
            else:
                return 0.0  # Tall robots cannot navigate in areas with low ceilings

        # Constraint: Uneven floors
        elif constraint == "uneven floors":
            if robot_mobility_type in ["tracked", "legged"]:
                score += 1.0  # These types handle uneven floors well

        # Constraint: Low visibility
        elif constraint == "low visibility":
            if robot_mobility_type in ["wheeled", "tracked", "legged"]:
                score += 1.0  # Infrared or LiDAR-equipped robots are suitable

        # Constraint: Slippery surfaces
        elif constraint == "slippery":
            if robot_mobility_type in ["tracked", "hovering", "aerial"]:
                score += 1.0  # Hovering and tracked types handle slippery surfaces better
            elif robot_mobility_type in ["wheeled", "legged"]:
                return 0.0  # Wheeled and legged robots are unsuitable on slippery floors

        # Constraint: Crowded environments
        elif constraint == "crowded":
            if robot_size[0] <= 1.0 and robot_size[1] <= 1.0:
                score += 1.0  # Smaller robots are more suitable in crowded environments
            else:
                score += 0.5  # Larger robots get a lower score

        # Constraint: Loose debris
        elif constraint == "loose debris":
            if robot_mobility_type in ["aerial", "hovering"]:
                score += 1.0  # Aerial and hovering robots handle debris better
            elif robot_mobility_type == "legged":
                return 0.0  # Legged robots are unsuitable

        # Constraint: No-fly zone
        elif constraint == "no-fly zone":
            if robot_mobility_type == "aerial":
                return 0.0  # Aerial robots cannot navigate in no-fly zones
            else:
                score += 0.5  # Other mobility types are unaffected

        # Constraint: Windy conditions
        elif constraint == "windy":
            if robot_mobility_type == "aerial":
                return 0.0  # Aerial robots struggle in windy conditions
            else:
                score += 0.5  # All other types are more stable in wind

        # Constraint: Dense obstructions (e.g., tree branches, hanging cables)
        elif constraint == "dense obstructions":
            if robot_mobility_type in ["aerial", "legged"]:
                return 0.0  # Aerial and legged robots are unsuitable in dense obstruction areas
            else:
                score += 0.5  # Other types may navigate dense areas on the ground

        # Constraint: Smooth floors
        elif constraint == "smooth surfaces":
            if robot_mobility_type == "climbing":
                return 0.0  # Climbing robots are less suited for smooth surfaces

    # Final suitability score (0 if any constraint returns 0)
    return score if score > 0 else 0.0


def evaluate_suitability_loose(robot: CapabilityProfile, task: TaskDescription) -> float:
    """
    Evaluates the suitability of a robot for a given task.
    A higher score indicates better suitability.
    
    Parameters:
        robot: The CapabilityProfile of the robot.
        task: The TaskDescription of the task.
    
    Returns:
        A float score representing the suitability of the robot for the task. A score of 0 indicates the robot cannot perform the task.
    """
    score = 0.0
    
#     print(task.required_capabilities, robot.payload_capacity)
    # Check if robot meets the minimum requirements
    if any(req for req in task.required_capabilities if "payload capacity" in req and robot.payload_capacity < float(req.split(">= ")[-1])):
        score += 0.0  # Suitability is zero if the robot doesn't meet minimum requirements
    else:
        score += 1.0  # Add score if payload meets or exceeds requirements
    
#     print(task.tools_needed, robot.sensors+robot.manipulators)
    # Check if the robot has the necessary tools for the task
    if robot.sensors: 
        if not all(item in robot.sensors for item in task.tools_needed):
            score += 0.0  # Suitability is zero if the robot lacks necessary tools
        else:
            score += 1.0  # Add score if robot has necessary tools
    
    if robot.manipulators: 
        if not all(item in robot.manipulators for item in task.tools_needed):
            score += 0.0  # Suitability is zero if the robot lacks necessary tools
        else:
            score += 1.0  # Add score if robot has necessary tools

#     print(task.communication_requirements, robot.communication_protocols)
    # Check if the robot can communicate as required by the task
    if task.communication_requirements and not all(protocol in robot.communication_protocols for protocol in task.communication_requirements):
        score += 0.0  # Suitability is zero if the robot lacks required communication protocols
    else:
        score += 1.0  # Add score if robot has communication requirements
    
#     print(task.safety_protocols, robot.safety_features)
    # Check if the robot can safely perform the task
    if robot.safety_features and task.safety_protocols:
        if task.safety_protocols and not all(safety in robot.safety_features for safety in task.safety_protocols):
            score += 0.0  # Suitability is zero if the robot lacks required safety features
        else:
            score += 1.0  # Add score if robot meets safety requirements
    
#     print(task.environmental_conditions, robot.environmental_resistance)
    # Environmental compatibility: Can the robot operate in the task’s conditions?
    if robot.environmental_resistance and task.environmental_conditions:
        if not all(condition in robot.environmental_resistance for condition in task.environmental_conditions):
            score += 0.0  # Suitability is zero if the robot can't operate in required environmental conditions
        else:
            score += 1.0  # Add score if robot has required environmental resistances
    
#     print(task.required_capabilities, robot.reach)
    # Check if the robot meets reach requirements
    if any(req for req in task.required_capabilities if "reach" in req and robot.reach < float(req.split(">= ")[-1])):
        score += 0.0  # Suitability is zero if the robot cannot reach the task area as required
    else:
        score += 1.0  # Add score if reach meets or exceeds requirements
    
#     print(task.navigation_constraints, robot.mobility_type, robot.size)
    # Check navigation constraints based on mobility type and robot size
    if task.navigation_constraints:
        navigation_match = navigation_suitability(robot.mobility_type, robot.size, task.navigation_constraints)
        if navigation_match == 0:
            score += 0.0
        else:
            score += navigation_match

    # NOTE: CHANGED TO WORK WITH COORDINATES
    # distance_to_task = ((robot.location[0] - task.location[0]) ** 2 + (robot.location[1] - task.location[1]) ** 2) ** 0.5
    distance_to_task = len(robot.current_path) - 1
#     print(robot.sensor_range)
    # Check sensor capabilities for the task
    if robot.sensor_range:
        if robot.sensor_range >= distance_to_task:
            score += 1.0
        elif robot.sensor_range >= distance_to_task/2:
            score += 0.5

    # Battery and distance check: Ensure the robot has sufficient battery to reach and complete the task
#     print(robot.max_speed, robot.battery_life, task.duration, distance_to_task)
    if ((distance_to_task / robot.max_speed)+task.time_to_complete) > robot.battery_life:
        score += 0.0  # Suitability is zero if the robot can't complete the task due to distance, speed, or battery limitations

    # Add to score based on proximity (closer robots get higher scores)
    if distance_to_task < 20.0:
        score += 1.0
    elif distance_to_task < 50.0:
        score += 0.5

#     print(task.priority_level, robot.autonomy_level)
    # Check if the robot's autonomy level matches the task's priority level
    if task.priority_level in ["high", "urgent"] and robot.autonomy_level in ["fully autonomous", "teleoperated"]:
        score += 1.0
    elif task.priority_level in ["medium", "low"] and robot.autonomy_level in ["semi-autonomous", "fully autonomous"]:
        score += 0.5

#     print(robot.battery_life, task.duration)
    # Evaluate battery life for task duration
    if robot.battery_life >= 2*((distance_to_task / robot.max_speed)+task.time_to_complete):
        score += 1.0
    else:
        score += 0.5

#     print(task.task_type, robot.special_functions)
    task_function_mapping = {
        "delivery": ["object recognition", "speech output", "facial recognition"],
        "inspection": ["object recognition", "object tracking", "gesture recognition"],
        "cleaning": ["object recognition"],
        "monitoring": ["speech output", "object tracking", "facial recognition"],
        "maintenance": ["object recognition", "path planning"],
        "assembly": ["object recognition"],
        "surveying": ["speech output", "facial recognition", "object recognition", "object tracking"],
        "data collection": ["object recognition", "object tracking", "facial recognition", "gesture recognition"],
        "assistance": ["speech output", "facial recognition", "gesture recognition"]
    }

    # Get the relevant functions for this task type
    required_functions = task_function_mapping[task.task_type]

    # Calculate the score based on matches between robot's functions and required functions
    if robot.special_functions:
        for function in robot.special_functions:
            if function in required_functions:
                score += 1.0  # Increase score for each match
        
#     # Dependencies
#     if task.dependencies:
#         # Assume dependencies are represented as tasks that must be completed first
#         score += 0.5 if all(dep in completed_tasks for dep in task.dependencies) else 0.0
    
#     print(task.difficulty, robot.processing_power)
    # Processing power: Certain tasks may benefit from higher processing power if they are computationally demanding
    if task.difficulty > 7 and robot.processing_power >= 5.0:  # Difficulty > 7 indicates a complex task
        score += 1.0
    elif task.difficulty > 4 and robot.processing_power >= 3.0:
        score += 1.0
    elif task.difficulty > 2 and robot.processing_power >= 1.5:
        score += 0.5

#     print(robot.adaptability)
    # Consider robot's adaptability to changing conditions
    if robot.adaptability:
        score += 0.5
    
#     print(task.task_type, robot.preferred_tasks)
    # Preference matching
    #if task.task_type in robot.preferred_tasks:
    #    score += 1.0

    # Score based on priority, reward, and difficulty
    priority_multiplier = {"low": 0.5, "medium": 1.0, "high": 1.5, "urgent": 2.0}[task.priority_level]
    reward_to_difficulty_ratio = task.reward / task.difficulty
#     print(task.priority_level, task.reward, task.difficulty, priority_multiplier, reward_to_difficulty_ratio)
    score += priority_multiplier * reward_to_difficulty_ratio

    # Return the final suitability score
#     print(score)
    return score

def evaluate_suitability_strict(robot: CapabilityProfile, task: TaskDescription) -> float:
    """
    Evaluates the suitability of a robot for a given task.
    A higher score indicates better suitability.
    
    Parameters:
        robot: The CapabilityProfile of the robot.
        task: The TaskDescription of the task.
    
    Returns:
        A float score representing the suitability of the robot for the task. A score of 0 indicates the robot cannot perform the task.
    """
    score = 0.0
    
#     print(task.required_capabilities, robot.payload_capacity)
    # Check if robot meets the minimum requirements
    if any(req for req in task.required_capabilities if "payload capacity" in req and robot.payload_capacity < float(req.split(">= ")[-1])):
        return 0.0  # Suitability is zero if the robot doesn't meet minimum requirements
    else:
        score += 1.0  # Add score if payload meets or exceeds requirements
    
#     print(task.tools_needed, robot.sensors+robot.manipulators)
    # Check if the robot has the necessary tools for the task
    if robot.sensors:
        if task.tools_needed and not all(item in robot.sensors for item in task.tools_needed):
            return 0.0  # Suitability is zero if the robot lacks necessary tools
        else:
            score += 1.0  # Add score if robot has necessary tools
    
    if robot.manipulators:
        if task.tools_needed and not all(item in robot.manipulators for item in task.tools_needed):
            return 0.0  # Suitability is zero if the robot lacks necessary tools
        else:
            score += 1.0  # Add score if robot has necessary tools
    
#     print(task.communication_requirements, robot.communication_protocols)
    # Check if the robot can communicate as required by the task
    if task.communication_requirements and not all(protocol in robot.communication_protocols for protocol in task.communication_requirements):
        return 0.0  # Suitability is zero if the robot lacks required communication protocols
    else:
        score += 1.0  # Add score if robot has communication requirements
    
#     print(task.safety_protocols, robot.safety_features)
    # Check if the robot can safely perform the task
    if robot.safety_features and task.safety_protocols:
        if task.safety_protocols and not all(safety in robot.safety_features for safety in task.safety_protocols):
            return 0.0  # Suitability is zero if the robot lacks required safety features
        else:
            score += 1.0  # Add score if robot meets safety requirements
    
#     print(task.environmental_conditions, robot.environmental_resistance)
    # Environmental compatibility: Can the robot operate in the task’s conditions?
    if robot.environmental_resistance and task.environmental_conditions:
        if task.environmental_conditions and not all(condition in robot.environmental_resistance for condition in task.environmental_conditions):
            return 0.0  # Suitability is zero if the robot can't operate in required environmental conditions
        else:
            score += 1.0  # Add score if robot has required environmental resistances
    
#     print(task.required_capabilities, robot.reach)
    # Check if the robot meets reach requirements
    if any(req for req in task.required_capabilities if "reach" in req and robot.reach < float(req.split(">= ")[-1])):
        return 0.0  # Suitability is zero if the robot cannot reach the task area as required
    else:
        score += 1.0  # Add score if reach meets or exceeds requirements
    
#     print(task.navigation_constraints, robot.mobility_type, robot.size)
    # Check navigation constraints based on mobility type and robot size
    if task.navigation_constraints:
        navigation_match = navigation_suitability(robot.mobility_type, robot.size, task.navigation_constraints)
        if navigation_match == 0:
            return 0.0
        else:
            score += navigation_match

    # NOTE: CHANGED TO WORK WITH COORDINATES
    # distance_to_task = ((robot.location[0] - task.location[0]) ** 2 + (robot.location[1] - task.location[1]) ** 2) ** 0.5
    distance_to_task = len(robot.current_path) - 1
#     print(robot.sensor_range)
    # Check sensor capabilities for the task
    if robot.sensor_range:
        if robot.sensor_range >= distance_to_task:
            score += 1.0
        elif robot.sensor_range >= distance_to_task/2:
            score += 0.5

    # Battery and distance check: Ensure the robot has sufficient battery to reach and complete the task
#     print(robot.max_speed, robot.battery_life, task.duration, distance_to_task)
    if ((distance_to_task / robot.max_speed)+task.time_to_complete) > robot.battery_life:
        return 0.0  # Suitability is zero if the robot can't complete the task due to distance, speed, or battery limitations

    # Add to score based on proximity (closer robots get higher scores)
    if distance_to_task < 20.0:
        score += 1.0
    elif distance_to_task < 50.0:
        score += 0.5

#     print(task.priority_level, robot.autonomy_level)
    # Check if the robot's autonomy level matches the task's priority level
    if task.priority_level in ["high", "urgent"] and robot.autonomy_level in ["fully autonomous", "teleoperated"]:
        score += 1.0
    elif task.priority_level in ["medium", "low"] and robot.autonomy_level in ["semi-autonomous", "fully autonomous"]:
        score += 0.5

#     print(robot.battery_life, task.duration)
    # Evaluate battery life for task duration
    if robot.battery_life >= 2*((distance_to_task / robot.max_speed)+task.time_to_complete):
        score += 1.0
    else:
        score += 0.5

#     print(task.task_type, robot.special_functions)
    task_function_mapping = {
        "delivery": ["object recognition", "speech output", "facial recognition"],
        "inspection": ["object recognition", "object tracking", "gesture recognition"],
        "cleaning": ["object recognition"],
        "monitoring": ["speech output", "object tracking", "facial recognition"],
        "maintenance": ["object recognition", "path planning"],
        "assembly": ["object recognition"],
        "surveying": ["speech output", "facial recognition", "object recognition", "object tracking"],
        "data collection": ["object recognition", "object tracking", "facial recognition", "gesture recognition"],
        "assistance": ["speech output", "facial recognition", "gesture recognition"]
    }

    # Get the relevant functions for this task type
    required_functions = task_function_mapping[task.task_type]

    # Calculate the score based on matches between robot's functions and required functions
    if robot.special_functions:
        for function in robot.special_functions:
            if function in required_functions:
                score += 1.0  # Increase score for each match
    
#     # Dependencies
#     if task.dependencies:
#         # Assume dependencies are represented as tasks that must be completed first
#         score += 0.5 if all(dep in completed_tasks for dep in task.dependencies) else 0.0
    
#     print(task.difficulty, robot.processing_power)
    # Processing power: Certain tasks may benefit from higher processing power if they are computationally demanding
    if task.difficulty > 7 and robot.processing_power >= 5.0:  # Difficulty > 7 indicates a complex task
        score += 1.0
    elif task.difficulty > 4 and robot.processing_power >= 3.0:
        score += 1.0
    elif task.difficulty > 2 and robot.processing_power >= 1.5:
        score += 0.5

#     print(robot.adaptability)
    # Consider robot's adaptability to changing conditions
    if robot.adaptability:
        score += 0.5
    
#     print(task.task_type, robot.preferred_tasks)
    # Preference matching
    #if task.task_type in robot.preferred_tasks:
    #    score += 1.0

    # Score based on priority, reward, and difficulty
    priority_multiplier = {"low": 0.5, "medium": 1.0, "high": 1.5, "urgent": 2.0}[task.priority_level]
    reward_to_difficulty_ratio = task.reward / task.difficulty
#     print(task.priority_level, task.reward, task.difficulty, priority_multiplier, reward_to_difficulty_ratio)
    score += priority_multiplier * reward_to_difficulty_ratio

    # Return the final suitability score
#     print(score)
    return score

def evaluate_suitability_distance(robot: CapabilityProfile, task: TaskDescription) -> float:
    """
    Evaluates the suitability of a robot for a given task.
    A higher score indicates better suitability.
    
    Parameters:
        robot: The CapabilityProfile of the robot.
        task: The TaskDescription of the task.
    
    Returns:
        A float score representing the suitability of the robot for the task. A score of 0 indicates the robot cannot perform the task.
    """
    score = 0.0
    
#     print(task.required_capabilities, robot.payload_capacity)
    # Check if robot meets the minimum requirements
    if any(req for req in task.required_capabilities if "payload capacity" in req and robot.payload_capacity < float(req.split(">= ")[-1])):
        score += 0.0  # Suitability is zero if the robot doesn't meet minimum requirements
    else:
        score += 1.0  # Add score if payload meets or exceeds requirements
    
#     print(task.tools_needed, robot.sensors+robot.manipulators)
    # Check if the robot has the necessary tools for the task
    if robot.sensors:
        if task.tools_needed and not all(item in robot.sensors for item in task.tools_needed):
            score += 0.0  # Suitability is zero if the robot lacks necessary tools
        else:
            score += 1.0  # Add score if robot has necessary tools

    if robot.manipulators:
        if task.tools_needed and not all(item in robot.manipulators for item in task.tools_needed):
            score += 0.0  # Suitability is zero if the robot lacks necessary tools
        else:
            score += 1.0  # Add score if robot has necessary tools
    
#     print(task.communication_requirements, robot.communication_protocols)
    # Check if the robot can communicate as required by the task
    if task.communication_requirements and not all(protocol in robot.communication_protocols for protocol in task.communication_requirements):
        score += 0.0  # Suitability is zero if the robot lacks required communication protocols
    else:
        score += 1.0  # Add score if robot has communication requirements
    
#     print(task.safety_protocols, robot.safety_features)
    # Check if the robot can safely perform the task
    if robot.safety_features and task.safety_protocols:
        if task.safety_protocols and not all(safety in robot.safety_features for safety in task.safety_protocols):
            score += 0.0  # Suitability is zero if the robot lacks required safety features
        else:
            score += 1.0  # Add score if robot meets safety requirements
    
#     print(task.environmental_conditions, robot.environmental_resistance)
    # Environmental compatibility: Can the robot operate in the task’s conditions?
    if robot.environmental_resistance and task.environmental_conditions:
        if task.environmental_conditions and not all(condition in robot.environmental_resistance for condition in task.environmental_conditions):
            score += 0.0  # Suitability is zero if the robot can't operate in required environmental conditions
        else:
            score += 1.0  # Add score if robot has required environmental resistances
    
#     print(task.required_capabilities, robot.reach)
    # Check if the robot meets reach requirements
    if any(req for req in task.required_capabilities if "reach" in req and robot.reach < float(req.split(">= ")[-1])):
        score += 0.0  # Suitability is zero if the robot cannot reach the task area as required
    else:
        score += 1.0  # Add score if reach meets or exceeds requirements
    
#     print(task.navigation_constraints, robot.mobility_type, robot.size)
    # Check navigation constraints based on mobility type and robot size
    if task.navigation_constraints:
        navigation_match = navigation_suitability(robot.mobility_type, robot.size, task.navigation_constraints)
        if navigation_match == 0:
            score += 0.0
        else:
            score += navigation_match

    # NOTE: CHANGED TO WORK WITH COORDINATES
    # distance_to_task = ((robot.location[0] - task.location[0]) ** 2 + (robot.location[1] - task.location[1]) ** 2) ** 0.5
    # stop suitability matrix from going negative
    distance_to_task = max(0, len(robot.current_path) - 1)
#     print(robot.sensor_range)
    # Check sensor capabilities for the task
    if robot.sensor_range:
        if robot.sensor_range >= distance_to_task:
            score += 1.0
        elif robot.sensor_range >= distance_to_task/2:
            score += 0.5

    # Battery and distance check: Ensure the robot has sufficient battery to reach and complete the task
#     print(robot.max_speed, robot.battery_life, task.duration, distance_to_task)
    if ((distance_to_task / robot.max_speed)+task.time_to_complete) > robot.battery_life:
        score += 0.0  # Suitability is zero if the robot can't complete the task due to distance, speed, or battery limitations

    # Add to score based on proximity (closer robots get higher scores)
#     if distance_to_task < 20.0:
#         score += 1.0
#     elif distance_to_task < 50.0:
#         score += 0.5

#     print(task.priority_level, robot.autonomy_level)
    # Check if the robot's autonomy level matches the task's priority level
    if task.priority_level in ["high", "urgent"] and robot.autonomy_level in ["fully autonomous", "teleoperated"]:
        score += 1.0
    elif task.priority_level in ["medium", "low"] and robot.autonomy_level in ["semi-autonomous", "fully autonomous"]:
        score += 0.5

#     print(robot.battery_life, task.duration)
    # Evaluate battery life for task duration
    if robot.battery_life >= 2*((distance_to_task / robot.max_speed)+task.time_to_complete):
        score += 1.0
    else:
        score += 0.5

#     print(task.task_type, robot.special_functions)
    task_function_mapping = {
        "delivery": ["object recognition", "speech output", "facial recognition"],
        "inspection": ["object recognition", "object tracking", "gesture recognition"],
        "cleaning": ["object recognition"],
        "monitoring": ["speech output", "object tracking", "facial recognition"],
        "maintenance": ["object recognition", "path planning"],
        "assembly": ["object recognition"],
        "surveying": ["speech output", "facial recognition", "object recognition", "object tracking"],
        "data collection": ["object recognition", "object tracking", "facial recognition", "gesture recognition"],
        "assistance": ["speech output", "facial recognition", "gesture recognition"]
    }

    # Get the relevant functions for this task type
    required_functions = task_function_mapping[task.task_type]

    # Calculate the score based on matches between robot's functions and required functions
    if robot.special_functions:
        for function in robot.special_functions:
            if function in required_functions:
                score += 1.0  # Increase score for each match
    
#     # Dependencies
#     if task.dependencies:
#         # Assume dependencies are represented as tasks that must be completed first
#         score += 0.5 if all(dep in completed_tasks for dep in task.dependencies) else 0.0
    
#     print(task.difficulty, robot.processing_power)
    # Processing power: Certain tasks may benefit from higher processing power if they are computationally demanding
    if task.difficulty > 7 and robot.processing_power >= 5.0:  # Difficulty > 7 indicates a complex task
        score += 1.0
    elif task.difficulty > 4 and robot.processing_power >= 3.0:
        score += 1.0
    elif task.difficulty > 2 and robot.processing_power >= 1.5:
        score += 0.5

#     print(robot.adaptability)
    # Consider robot's adaptability to changing conditions
    if robot.adaptability:
        score += 0.5
    
#     print(task.task_type, robot.preferred_tasks)
    # Preference matching
    #if task.task_type in robot.preferred_tasks:
    #    score += 1.0

    # Score based on priority, reward, and difficulty
    priority_multiplier = {"low": 0.5, "medium": 1.0, "high": 1.5, "urgent": 2.0}[task.priority_level]
    reward_to_difficulty_ratio = task.reward / task.difficulty
#     print(task.priority_level, task.reward, task.difficulty, priority_multiplier, reward_to_difficulty_ratio)
    score += priority_multiplier * reward_to_difficulty_ratio

    # Weight score by distance to task
    # NOTE: IF THE ROBOT IS AT THE TASK THIS CAN CAUSE A DIVIDE BY ZERO ERROR
    score = score / (distance_to_task + 1E-5)
    
    # Return the final suitability score
#     print(score)
    return score

def evaluate_suitability_priority(robot: CapabilityProfile, task: TaskDescription) -> float:
    """
    Evaluates the suitability of a robot for a given task.
    A higher score indicates better suitability.
    
    Parameters:
        robot: The CapabilityProfile of the robot.
        task: The TaskDescription of the task.
    
    Returns:
        score: A float score representing the suitability of the robot for the task. A score of 0 indicates the robot cannot perform the task.
    """
    score = 0.0
    
#     print(task.required_capabilities, robot.payload_capacity)
    # Check if robot meets the minimum requirements
    if any(req for req in task.required_capabilities if "payload capacity" in req and robot.payload_capacity < float(req.split(">= ")[-1])):
        score += 0.0  # Suitability is zero if the robot doesn't meet minimum requirements
    else:
        score += 1.0  # Add score if payload meets or exceeds requirements
    
#     print(task.tools_needed, robot.sensors+robot.manipulators)
    # Check if the robot has the necessary tools for the task
    if robot.sensors:
        if task.tools_needed and not all(item in robot.sensors for item in task.tools_needed):
            score += 0.0  # Suitability is zero if the robot lacks necessary tools
        else:
            score += 1.0  # Add score if robot has necessary tools
    
    if robot.manipulators:
        if task.tools_needed and not all(item in robot.manipulators for item in task.tools_needed):
            score += 0.0  # Suitability is zero if the robot lacks necessary tools
        else:
            score += 1.0  # Add score if robot has necessary tools
    
#     print(task.communication_requirements, robot.communication_protocols)
    # Check if the robot can communicate as required by the task
    if task.communication_requirements and not all(protocol in robot.communication_protocols for protocol in task.communication_requirements):
        score += 0.0  # Suitability is zero if the robot lacks required communication protocols
    else:
        score += 1.0  # Add score if robot has communication requirements
    
#     print(task.safety_protocols, robot.safety_features)
    # Check if the robot can safely perform the task
    if robot.safety_features and task.safety_protocols:
        if task.safety_protocols and not all(safety in robot.safety_features for safety in task.safety_protocols):
            score += 0.0  # Suitability is zero if the robot lacks required safety features
        else:
            score += 1.0  # Add score if robot meets safety requirements
    
#     print(task.environmental_conditions, robot.environmental_resistance)
    # Environmental compatibility: Can the robot operate in the task’s conditions?
    if robot.environmental_resistance and task.environmental_conditions:
        if task.environmental_conditions and not all(condition in robot.environmental_resistance for condition in task.environmental_conditions):
            score += 0.0  # Suitability is zero if the robot can't operate in required environmental conditions
        else:
            score += 1.0  # Add score if robot has required environmental resistances
    
#     print(task.required_capabilities, robot.reach)
    # Check if the robot meets reach requirements
    if any(req for req in task.required_capabilities if "reach" in req and robot.reach < float(req.split(">= ")[-1])):
        score += 0.0  # Suitability is zero if the robot cannot reach the task area as required
    else:
        score += 1.0  # Add score if reach meets or exceeds requirements
    
#     print(task.navigation_constraints, robot.mobility_type, robot.size)
    # Check navigation constraints based on mobility type and robot size
    if task.navigation_constraints:
        navigation_match = navigation_suitability(robot.mobility_type, robot.size, task.navigation_constraints)
        if navigation_match == 0:
            score += 0.0
        else:
            score += navigation_match

    # NOTE: CHANGED TO WORK WITH COORDINATES
    # distance_to_task = ((robot.location[0] - task.location[0]) ** 2 + (robot.location[1] - task.location[1]) ** 2) ** 0.5
    distance_to_task = len(robot.current_path) - 1
#     print(robot.sensor_range)
    # Check sensor capabilities for the task
    if robot.sensor_range:
        if robot.sensor_range >= distance_to_task:
            score += 1.0
        elif robot.sensor_range >= distance_to_task/2:
            score += 0.5

    # Battery and distance check: Ensure the robot has sufficient battery to reach and complete the task
#     print(robot.max_speed, robot.battery_life, task.duration, distance_to_task)
    if ((distance_to_task / robot.max_speed)+task.time_to_complete) > robot.battery_life:
        score += 0.0  # Suitability is zero if the robot can't complete the task due to distance, speed, or battery limitations

    # Add to score based on proximity (closer robots get higher scores)
    if distance_to_task < 20.0:
        score += 1.0
    elif distance_to_task < 50.0:
        score += 0.5

#     print(task.priority_level, robot.autonomy_level)
    # Check if the robot's autonomy level matches the task's priority level
    if task.priority_level in ["high", "urgent"] and robot.autonomy_level in ["fully autonomous", "teleoperated"]:
        score += 1.0
    elif task.priority_level in ["medium", "low"] and robot.autonomy_level in ["semi-autonomous", "fully autonomous"]:
        score += 0.5

#     print(robot.battery_life, task.duration)
    # Evaluate battery life for task duration
    if robot.battery_life >= 2*((distance_to_task / robot.max_speed)+task.time_to_complete):
        score += 1.0
    else:
        score += 0.5

#     print(task.task_type, robot.special_functions)
    task_function_mapping = {
        "delivery": ["object recognition", "speech output", "facial recognition"],
        "inspection": ["object recognition", "object tracking", "gesture recognition"],
        "cleaning": ["object recognition"],
        "monitoring": ["speech output", "object tracking", "facial recognition"],
        "maintenance": ["object recognition", "path planning"],
        "assembly": ["object recognition"],
        "surveying": ["speech output", "facial recognition", "object recognition", "object tracking"],
        "data collection": ["object recognition", "object tracking", "facial recognition", "gesture recognition"],
        "assistance": ["speech output", "facial recognition", "gesture recognition"]
    }

    # Get the relevant functions for this task type
    required_functions = task_function_mapping[task.task_type]

    # Calculate the score based on matches between robot's functions and required functions
    if robot.special_functions:
        for function in robot.special_functions:
            if function in required_functions:
                score += 1.0  # Increase score for each match
    
#     # Dependencies
#     if task.dependencies:
#         # Assume dependencies are represented as tasks that must be completed first
#         score += 0.5 if all(dep in completed_tasks for dep in task.dependencies) else 0.0
    
#     print(task.difficulty, robot.processing_power)
    # Processing power: Certain tasks may benefit from higher processing power if they are computationally demanding
    if task.difficulty > 7 and robot.processing_power >= 5.0:  # Difficulty > 7 indicates a complex task
        score += 1.0
    elif task.difficulty > 4 and robot.processing_power >= 3.0:
        score += 1.0
    elif task.difficulty > 2 and robot.processing_power >= 1.5:
        score += 0.5

#     print(robot.adaptability)
    # Consider robot's adaptability to changing conditions
    if robot.adaptability:
        score += 0.5
    
#     print(task.task_type, robot.preferred_tasks)
    # Preference matching
    #if task.task_type in robot.preferred_tasks:
    #    score += 1.0

    # Score based on reward and difficulty
    priority_multiplier = {"low": 0.5, "medium": 1.0, "high": 1.5, "urgent": 2.0}[task.priority_level]
    reward_to_difficulty_ratio = task.reward / task.difficulty
#     print(task.priority_level, task.reward, task.difficulty, priority_multiplier, reward_to_difficulty_ratio)
    score += reward_to_difficulty_ratio

    # Weight based on priority
    score = score * priority_multiplier
    
    # Return the final suitability score
#     print(score)
    return score


In [7]:
def evaluate_suitability_with_llm(robot: CapabilityProfile, task: TaskDescription) -> str:
    """
    Uses a large language model (LLM) to evaluate the suitability of a robot for a given task.

    Parameters:
        robot: The CapabilityProfile of the robot.
        task: The TaskDescription of the task.

    Returns:
        score: A float score (0 to 10) representing the suitability of the robot for the task.
    """
    # Construct a detailed prompt describing the scenario
    prompt = f"""
    You are evaluating the suitability of a robot for a specific task in a robotic system.

    Here are the details of the robot:
    - Mobility Type: {robot.mobility_type}
    - Maximum Speed: {robot.max_speed} units/sec
    - Payload Capacity: {robot.payload_capacity} kg
    - Reach: {robot.reach} meters
    - Battery Life: {robot.battery_life} hours
    - Size: {robot.size} (length, width, height in meters)
    - Environmental Resistance: {', '.join(robot.environmental_resistance) if robot.environmental_resistance  else "No environmental resistance"}
    - Sensors: {', '.join(robot.sensors) if robot.sensors else "No sensors"}
    - Sensor Range: {robot.sensor_range if robot.sensor_range else "0"} meters
    - Manipulators: {', '.join(robot.manipulators) if robot.manipulators else "No manipulators"}
    - Communication Protocols: {', '.join(robot.communication_protocols)}
    - Processing Power: {robot.processing_power} units
    - Autonomy Level: {robot.autonomy_level}
    - Special Functions: {', '.join(robot.special_functions) if robot.special_functions else "No special functions"}
    - Safety Features: {', '.join(robot.safety_features) if robot.safety_features else "No safety features"}
    - Adaptability: {"Yes" if robot.adaptability else "No"}
    - Location: {robot.location[0]}, {robot.location[1]}

    Here are the details of the task:
    - Task Type: {task.task_type}
    - Objective: {task.objective}
    - Priority Level: {task.priority_level}
    - Reward: {task.reward} units
    - Difficulty: {task.difficulty} (scale of 1 to 10)
    - Location: {task.location} (x, y, z coordinates)
    - Navigation Constraints: {', '.join(task.navigation_constraints) if task.navigation_constraints else 'None'}
    - Required Capabilities: {', '.join(task.required_capabilities)}
    - Duration: {task.time_to_complete} seconds
    - Environmental Conditions: {', '.join(task.environmental_conditions) if task.environmental_conditions else 'None'}
    - Tools Needed: {', '.join(task.tools_needed) if task.tools_needed else 'None'}
    - Communication Requirements: {', '.join(task.communication_requirements) if task.communication_requirements else 'None'}
    - Safety Protocols: {', '.join(task.safety_protocols) if task.safety_protocols else 'None'}
    - Performance Metrics: {', '.join(task.performance_metrics) if task.performance_metrics else 'None'}
    - Success Criteria: {task.success_criteria}

    Based on the robot's capabilities and the task requirements, please rate the suitability of this robot for this task on a scale of 0 to 10, where 0 means the robot is completely unsuitable and 10 means it is perfectly suited.

    Provide a single number between 0 and 10 as the output score: 
    """

    # Send the prompt to the LLM and parse the response
    try:
        response = openai.Completion.create(
            engine="text-davinci-003",  # Replace with the appropriate model
            prompt=prompt,
            max_tokens=5,
            temperature=0  # Lower temperature for more deterministic responses
        )
        score_text = response.choices[0].text.strip()
        score = float(score_text) if score_text.isnumeric() else 0.0
    except Exception as e:
        print(f"Error querying LLM: {e}")
        score = 0.0  # Default to zero in case of an error

    return score

In [12]:
def generate_random_assignments(num_robots: int, num_tasks: int, num_assignments: int = 5) -> List[List[Tuple[Optional[int], Optional[int]]]]:
    """
    Generates a list of random assignments of robots to tasks.
    Each assignment is a list of (robot, task) pairs, where some robots or tasks may be unassigned.
    These assignments are not generated based on suitability scores.
    
    Parameters:
        num_robots: Number of available robots.
        num_tasks: Number of tasks to be assigned.
        num_assignments: Number of different random assignments to generate.
    
    Returns:
        assignments: A list of random assignments, where each assignment is a list of (robot, task) pairs.
    """
    assignments = []

    # Generate random assignments
    for _ in range(num_assignments):
        # Generate random indices for pairing
        available_robots = random.sample(range(num_robots), num_robots)
        available_tasks = random.sample(range(num_tasks), num_tasks)
        
        # Determine the number of pairs to create (limited by the smaller of num_robots or num_tasks)
        num_pairs = min(num_robots, num_tasks)

        # Pair robots and tasks using the precomputed random indices
        assigned_pairs = [(available_robots[i], available_tasks[i]) for i in range(num_pairs)]

        # Determine unassigned robots and tasks
        unassigned_robots = available_robots[num_pairs:] if num_robots > num_pairs else []
        unassigned_tasks = available_tasks[num_pairs:] if num_tasks > num_pairs else []

        assignments.append((assigned_pairs, unassigned_robots, unassigned_tasks))

    return assignments

def generate_all_unique_assignments(num_robots: int, num_tasks: int) -> List[Tuple[List[Tuple[int, int]], List[int], List[int]]]:
    """
    Generates all possible unique assignments of robots to tasks.
    Each assignment is a tuple containing:
      1. A list of (robot, task) pairs.
      2. A list of unassigned robots.
      3. A list of unassigned tasks.
    
    Parameters:
        num_robots: Number of available robots.
        num_tasks: Number of tasks to be assigned.
    
    Returns:
        all_assignments: A list of all possible unique assignments, where each assignment is a tuple (assigned_pairs, unassigned_robots, unassigned_tasks).
    """
    all_assignments = []

    # Determine the number of pairs to create (limited by the smaller of num_robots or num_tasks)
    num_pairs = min(num_robots, num_tasks)

    # Generate all possible combinations of robots for the tasks
    robot_combinations = list(combinations(range(num_robots), num_pairs))
    task_combinations = list(combinations(range(num_tasks), num_pairs))

    # Generate all permutations of those combinations
    for robot_comb in robot_combinations:
        for task_comb in task_combinations:
            # Create all permutations for the selected robot and task pairs
            for permuted_robots in permutations(robot_comb):
                for permuted_tasks in permutations(task_comb):
                    # Pair the robots and tasks
                    assigned_pairs = [(permuted_robots[i], permuted_tasks[i]) for i in range(num_pairs)]

                    # Determine unassigned robots and tasks
                    unassigned_robots = [r for r in range(num_robots) if r not in permuted_robots]
                    unassigned_tasks = [t for t in range(num_tasks) if t not in permuted_tasks]

                    all_assignments.append((assigned_pairs, unassigned_robots, unassigned_tasks))

    return all_assignments

def generate_high_suitability_assignments(num_robots: int, num_tasks: int, suitability_matrix: List[List[float]], num_assignments: int = 5) -> List[Tuple[List[Tuple[int, int]], List[int], List[int]]]:
    """
    Generates assignments for robots with the highest total suitability across all tasks.
    Each assignment includes the top robots paired with tasks based on suitability.
    
    Parameters:
        num_robots: Number of available robots.
        num_tasks: Number of tasks to be assigned.
        suitability_matrix: A 2D list where element [i][j] represents the suitability of robot i for task j.
        num_assignments: Number of different random assignments to generate.
    
    Returns:
        assignments: A list of high-suitability assignments, where each assignment is a tuple containing:
      - A list of (robot, task) pairs.
      - A list of unassigned robots.
      - A list of unassigned tasks.
    """
    assignments = []
    if num_robots == 1 and num_tasks == 1:
        assigned_pairs = [(0,0)]
        unassigned_robots = []
        unassigned_tasks = []
        assignments.append((assigned_pairs, unassigned_robots, unassigned_tasks))
        return assignments
    elif num_robots == 1:
        for task_id in range(num_tasks):
            assigned_pairs = [(0, task_id)]
            unassigned_tasks = list(set(range(num_tasks)) - {task_id})
            assignments.append((assigned_pairs, [], unassigned_tasks))
        return assignments
    elif num_tasks == 1:
        for robot_id in range(num_robots):
            assigned_pairs = [(robot_id, 0)]
            unassigned_robots = list(set(range(num_robots)) - {robot_id})
            assignments.append((assigned_pairs, unassigned_robots, []))
        return assignments
        
    num_assignments = min(num_assignments, num_robots * num_tasks * 10)
    

    # Calculate total suitability score for each robot across all tasks
    robot_suitability_scores = [(i, sum(suitability_matrix[i])) for i in range(num_robots)]
    
    # Sort robots by total suitability score in descending order
    sorted_robots = sorted(robot_suitability_scores, key=lambda x: x[1], reverse=True)
    
    # Select the top robots based on total suitability scores
    top_robot_indices = [robot[0] for robot in sorted_robots[:min(num_robots, num_tasks)]]
    
    # Identify robots with the maximum score for each task
    for task in range(num_tasks):
        max_score = max(suitability_matrix[robot][task] for robot in range(num_robots))
        top_task_robots = [robot for robot in range(num_robots) if suitability_matrix[robot][task] == max_score]
        top_robot_indices.extend(top_task_robots)
    
    # Remove duplicates and keep only as many robots as tasks if robots exceed tasks
    top_robot_indices = list(set(top_robot_indices))[:min(num_robots, num_tasks)]

    for _ in range(num_assignments):
        # Randomly sample tasks to pair with the top robots
        available_tasks = random.sample(range(num_tasks), min(num_tasks, num_robots))
        
        # Randomly sample robots to pair with the tasks
        available_robots = random.sample(top_robot_indices, min(num_tasks, num_robots))

        # Pair each selected robot with a randomly chosen task
        assigned_pairs = [(available_robots[i], available_tasks[i]) for i in range(len(available_robots))]
        
        # Determine unassigned robots and tasks
        unassigned_robots = list(set(range(num_robots)) - set(available_robots))
        unassigned_tasks = list(set(range(num_tasks)) - set(available_tasks))
        # Store each assignment as a tuple
        assignments.append((assigned_pairs, unassigned_robots, unassigned_tasks))

    return assignments

In [13]:
def calculate_total_suitability(assignment: List[Tuple[int, int]], suitability_matrix: List[List[float]]) -> float:
    """
    Calculates the total suitability score for a given assignment.
    
    Parameters:
        assignment: A list of (robot, task) pairs representing the assignment.
        suitability_matrix: A 2D list where the element at [i][j] represents the suitability of robot i for task j.
    
    Returns:
        total_suitability: The total suitability score for the assignment.
    """
    total_suitability = 0.0
    
    # Sum the suitability ratings for each robot-task pair in the assignment
    for robot, task in assignment:
        total_suitability += suitability_matrix[robot][task]
    
    return total_suitability

def rank_assignments_range(assignments: List[List[Tuple[int, int]]], suitability_matrix: List[List[float]]) -> Tuple[List[float], List[int]]:
    """
    Ranks the given assignments based on their total suitability scores.
    
    Parameters:
        assignments: A list of assignments, where each assignment is a list of (robot, task) pairs.
        suitability_matrix: A 2D list where the element at [i][j] represents the suitability of robot i for task j.
    
    Returns:
        (total_scores, ranking): A tuple where
      1. A list of total suitability scores for each assignment.
      2. A list of indices representing the ranking of the assignments, where the first index corresponds to the
         assignment with the highest total score.
    """
    total_scores = []

    # Calculate total suitability scores for each assignment
    for assignment in assignments:
        total_suitability = sum(suitability_matrix[robot][task] for robot, task in assignment[0])
        total_scores.append(total_suitability)

    # Get the ranking of assignments based on the total scores (higher score is better)
    ranking = sorted(range(len(total_scores)), key=lambda i: total_scores[i], reverse=True)
    
    return total_scores, ranking

def rank_assignments_borda(assignments: List[List[Tuple[int, int]]], suitability_matrix: List[List[float]]) -> Tuple[List[float], List[int]]:
    """
    Ranks each assignment from the perspective of each robot based on its suitability for its task in each assignment,
    and uses the Borda count method to aggregate these rankings.
    
    Parameters:
        assignments: A list of assignments, where each assignment is a list of (robot, task) pairs.
        suitability_matrix: A 2D list where the element at [i][j] represents the suitability of robot i for task j.
    
    Returns:
        (borda_scores, ranked_assignments): A tuple containing:
      1. A list of Borda scores for each assignment.
      2. A list of indices representing the ranking of the assignments based on Borda scores, where the first index corresponds
         to the assignment with the highest total Borda score.
    """
    num_assignments = len(assignments)
    num_robots = len(suitability_matrix)

    # Initialize Borda scores for each assignment
    borda_scores = [0] * num_assignments

    # For each robot, rank the assignments based on its suitability rating
    for robot in range(num_robots):
        # Collect suitability scores for this robot across all assignments
        robot_scores = []
        
        for assignment_index, assignment in enumerate(assignments):
            # Find the task assigned to this robot in the current assignment
            assigned_task = next((task for r, task in assignment[0] if r == robot), None)
            
            # If the robot has no task assigned in this assignment, assign it the lowest score
            if assigned_task is None:
                robot_scores.append((assignment_index, -1))  # Using -1 to rank unassigned lower
            else:
                robot_scores.append((assignment_index, suitability_matrix[robot][assigned_task]))

        # Sort robot's scores for all assignments: unassigned (score -1) at the bottom, then by suitability
        robot_scores.sort(key=lambda x: x[1], reverse=True)

        # Assign Borda points based on the ranking
        for rank, (assignment_index, _) in enumerate(robot_scores):
            borda_points = len(robot_scores) - rank - 1  # Borda count: higher ranks get more points
            borda_scores[assignment_index] += borda_points

    # Rank assignments based on total Borda scores (higher score is better)
    ranked_assignments = sorted(range(num_assignments), key=lambda i: borda_scores[i], reverse=True)

    return borda_scores, ranked_assignments

def rank_assignments_approval(assignments: List[List[Tuple[int, int]]], suitability_matrix: List[List[float]], threshold: float = 10) -> Tuple[List[int], List[int]]:
    """
    Uses approval voting where each robot gives 1 point to assignments where its suitability for its task is above a threshold.
    If a robot's suitability is below the threshold or it is unassigned, it gives 0 points for that assignment.
    
    Parameters:
        assignments: A list of assignments, where each assignment is a list of (robot, task) pairs.
        suitability_matrix: A 2D list where the element at [i][j] represents the suitability of robot i for task j.
        threshold: The suitability threshold above which a robot will approve an assignment (default is 10).
    
    Returns:
        (approval_scores, ranked_assignments): A tuple containing:
      1. A list of approval scores for each assignment.
      2. A list of indices representing the ranking of the assignments based on approval scores.
    """
    num_assignments = len(assignments)
    num_robots = len(suitability_matrix)

    # Initialize approval scores for each assignment
    approval_scores = [0] * num_assignments

    # For each robot, evaluate each assignment based on the suitability threshold
    for robot in range(num_robots):
        for assignment_index, assignment in enumerate(assignments):
            # Find the task assigned to this robot in the current assignment
            assigned_task = next((task for r, task in assignment[0] if r == robot), None)
            
            # Check if robot's suitability rating for this task is above the threshold
            if assigned_task  and suitability_matrix[robot][assigned_task] > threshold:
                approval_scores[assignment_index] += 1  # Robot approves this assignment

    # Rank assignments based on approval scores (higher score is better)
    ranked_assignments = sorted(range(num_assignments), key=lambda i: approval_scores[i], reverse=True)

    return approval_scores, ranked_assignments

def rank_assignments_majority_judgment(assignments: List[List[Tuple[int, int]]], suitability_matrix: List[List[float]]) -> Tuple[List[float], List[int]]:
    """
    Uses majority judgment to rank assignments based on robot suitability for assigned tasks.
    
    Parameters:
        assignments: A list of assignments, where each assignment is a list of (robot, task) pairs.
        suitability_matrix: A 2D list where the element at [i][j] represents the suitability of robot i for task j.
    
    Returns:
        (assignment_ratings, ranked_assignments): A tuple containing:
      1. A list of median scores for each assignment based on majority judgment.
      2. A list of indices representing the ranking of assignments based on median scores.
    """
    # Define qualitative rating categories mapped to suitability score ranges
    rating_scale = {"Excellent": 3, "Good": 2, "Fair": 1, "Poor": 0}
    rating_thresholds = [(15, "Excellent"), (10, "Good"), (5, "Fair"), (0, "Poor")]

    assignment_ratings = []

    for assignment in assignments:
        ratings = []
        
        for robot, task in assignment[0]:
            # Get the suitability score for the robot-task pair
            suitability_score = suitability_matrix[robot][task]

            # Convert suitability score to qualitative rating
            for threshold, rating in rating_thresholds:
                if suitability_score >= threshold:
                    ratings.append(rating_scale[rating])
                    break
        
        # Calculate the median rating for this assignment
        median_rating = np.median(ratings) if ratings else 0
        assignment_ratings.append(median_rating)

    # Rank assignments based on median ratings
    ranked_assignments = sorted(range(len(assignment_ratings)), key=lambda i: assignment_ratings[i], reverse=True)

    return assignment_ratings, ranked_assignments

def rank_assignments_cumulative_voting(assignments: List[List[Tuple[int, int]]], suitability_matrix: List[List[float]], total_votes: int = 10) -> Tuple[List[float], List[int]]:
    """
    Uses cumulative voting to rank assignments, where each robot distributes a fixed number of votes
    across assignments based on suitability scores.
    
    Parameters:
        assignments: A list of assignments, where each assignment is a list of (robot, task) pairs.
        suitability_matrix: A 2D list where the element at [i][j] represents the suitability of robot i for task j.
        total_votes: The total number of votes each robot has to distribute (default is 10).
    
    Returns:
        (cumulative_votes, ranked_assignments): A tuple containing:
      1. A list of cumulative votes for each assignment.
      2. A list of indices representing the ranking of assignments based on cumulative votes.
    """
    num_assignments = len(assignments)
    cumulative_votes = [0] * num_assignments
    if total_votes < num_assignments/2:
        total_votes = num_assignments/2
    
    for robot in range(len(suitability_matrix)):
        # Collect suitability scores for this robot across all assignments
        robot_votes = []
        
        for assignment_index, assignment in enumerate(assignments):
            # Find the task assigned to this robot in the current assignment
            assigned_task = next((task for r, task in assignment[0] if r == robot), None)
            
            # If robot is unassigned in this assignment, give it a suitability score of 0
            suitability_score = suitability_matrix[robot][assigned_task] if assigned_task is not None else 0
            robot_votes.append((assignment_index, suitability_score))
        
        # Sort robot votes based on suitability scores in descending order
        robot_votes.sort(key=lambda x: x[1], reverse=True)

        # Distribute the total votes among assignments proportional to suitability scores
        total_score = sum(score for _, score in robot_votes if score > 0)
        for assignment_index, score in robot_votes:
            if total_score > 0:
                cumulative_votes[assignment_index] += total_votes * (score / total_score)

    # Rank assignments based on cumulative votes (higher score is better)
    ranked_assignments = sorted(range(num_assignments), key=lambda i: cumulative_votes[i], reverse=True)

    return cumulative_votes, ranked_assignments

def rank_assignments_condorcet_method(assignments: List[List[Tuple[int, int]]], suitability_matrix: List[List[float]]) -> Tuple[List[int], List[int]]:
    """
    Uses the Condorcet method to rank assignments by comparing each assignment in a pairwise manner.
    
    Parameters:
        assignments: A list of assignments, where each assignment is a list of (robot, task) pairs.
        suitability_matrix: A 2D list where the element at [i][j] represents the suitability of robot i for task j.
    
    Returns:
        (pairwise_wins, ranked_assignments): A tuple containing:
      1. A list of pairwise win counts for each assignment.
      2. A list of indices representing the ranking of assignments based on pairwise win counts.
    """
    num_assignments = len(assignments)
    num_robots = len(suitability_matrix)

    # Initialize a matrix to track pairwise wins for each assignment comparison
    pairwise_wins = [0] * num_assignments

    # Compare each pair of assignments
    for i in range(num_assignments):
        for j in range(i + 1, num_assignments):
            # Count the number of robots that prefer assignment i over assignment j and vice versa
            i_wins, j_wins = 0, 0
            
            for robot in range(num_robots):
                # Find the task assigned to the robot in both assignments
                task_i = next((task for r, task in assignments[i][0] if r == robot), None)
                task_j = next((task for r, task in assignments[j][0] if r == robot), None)
                
                # Get suitability scores (or 0 if unassigned)
                suitability_i = suitability_matrix[robot][task_i] if task_i is not None else 0
                suitability_j = suitability_matrix[robot][task_j] if task_j is not None else 0

                # Determine which assignment the robot prefers
                if suitability_i > suitability_j:
                    i_wins += 1
                elif suitability_j > suitability_i:
                    j_wins += 1

            # Update pairwise win counts based on robot preferences
            if i_wins > j_wins:
                pairwise_wins[i] += 1
            elif j_wins > i_wins:
                pairwise_wins[j] += 1

    # Rank assignments based on pairwise win counts
    ranked_assignments = sorted(range(num_assignments), key=lambda k: pairwise_wins[k], reverse=True)

    return pairwise_wins, ranked_assignments

def check_zero_suitability(assignment: List[Tuple[int, int]], suitability_matrix: List[List[float]]) -> bool:
    """
    Checks if any robot-task pair in the assignment has a suitability rating of 0.
    
    Parameters:
        assignment: A list of (robot, task) pairs representing the assignment.
        suitability_matrix: A 2D list where the element at [i][j] represents the suitability of robot i for task j.
    
    Returns:
        Bool: True if any robot-task pair in the assignment has a suitability of 0, otherwise False.
    """
    for robot, task in assignment:
        if suitability_matrix[robot][task] == 0:
            return True  # Found a zero suitability rating
    
    return False  # No zero suitability ratings found

In [14]:
def cbba_task_allocation(suitability_matrix: List[List[float]]) -> List[Tuple[int, int]]:
    """
    Uses a two-phase Consensus-Based Bundle Algorithm (CBBA) for task allocation.
    Each robot is assigned to one task, and each task is assigned to one robot.
    
    Parameters:
        suitability_matrix: A 2D list where the element at [i][j] represents the suitability score of robot i for task j.
    
    Returns:
        final_assignment: A list of (robot, task) pairs representing the final allocation.
    """
    num_robots = len(suitability_matrix)
    num_tasks = len(suitability_matrix[0])

    # Initialize assignment variables
    robot_bundles = [[] for _ in range(num_robots)]  # Bundle of tasks for each robot
    task_bids = [-1] * num_tasks  # Highest bid for each task

    # Phase 1: Bundle Construction
    for robot in range(num_robots):
        # Each robot evaluates each task for inclusion in its bundle
        for task in range(num_tasks):
            bid = suitability_matrix[robot][task]
            if bid > task_bids[task]:
                task_bids[task] = bid
            robot_bundles[robot].append(task)
    
    not_assigned = list(range(num_robots))
    # Phase 2: Conflict Resolution
    updated = True
    while updated:
        updated = False
        
        for robot in range(num_robots):
            if not robot_bundles[robot] or robot not in not_assigned:
                continue  # Skip robots with no tasks in their bundle
                
            # Sort the robot's bundle in descending order of suitability scores
            robot_bundles[robot].sort(key=lambda task: suitability_matrix[robot][task], reverse=True)
            # Iterate through tasks in the robot's bundle
            for task in robot_bundles[robot][:]:
                # If the robot has the highest unique bid for this task, assign it
                highest_bid = max(suitability_matrix[competing_robot][task] for competing_robot in not_assigned)
                if suitability_matrix[robot][task] == highest_bid:
                    updated = True
                    task_bids[task] = suitability_matrix[robot][task]
                    # Clear remaining tasks from this robot's bundle
                    robot_bundles[robot] = [task]
                    not_assigned.remove(robot)
                    for r in range(num_robots):
                        if r != robot and task in robot_bundles[r]:
                            robot_bundles[r].remove(task)
                    break

        if len(not_assigned) == 0:
            updated = False
                    
    final_assignment = [(index, bundle[0]) for index, bundle in enumerate(robot_bundles) if bundle]

    return final_assignment

def ssia_task_allocation(suitability_matrix: List[List[float]]) -> List[Tuple[int, int]]:
    """
    Uses the Sequential Single-Item Auction (SSIA) for task allocation.
    
    Parameters:
        suitability_matrix: A 2D list where the element at [i][j] represents the suitability score of robot i for task j.
    
    Returns:
        assignments: A list of (robot, task) pairs representing the final allocation.
    """
    num_robots = len(suitability_matrix)
    num_tasks = len(suitability_matrix[0])
    print(f"Suitability Matrix Issue: {suitability_matrix}")
    print(f"Number of Robots: {num_robots}, Number of Tasks: {num_tasks}")

    
    # Initialize assignment list to store (robot, task) pairs
    assignments = []
    assigned_robots = set()  # Track robots that have already been assigned
    
    # Auction tasks sequentially
    for task in range(num_tasks):
        # NOTE: I can fix it by making the highest_bid negative infinity but I want to see why the suitability matrix is all negative
        highest_bid = -1
        winning_robot = -1

        # Robots bid for the current task
        for robot in range(num_robots):
            if robot in assigned_robots:
                continue  # Skip if the robot is already assigned
            bid = suitability_matrix[robot][task]
            if bid > highest_bid:
                # check if the bid is positive because we are getting an empty assigned robots dictionary
                print(f"Robot {robot} bids {bid} for Task {task}")
                highest_bid = bid
                winning_robot = robot
        
        # Assign the task to the robot with the highest bid if any bid is positive
        # if highest_bid >= 0:
        if winning_robot != -1:
            assignments.append((winning_robot, task))
            assigned_robots.add(winning_robot)  # Mark this robot as assigned
    
    return assignments

def ilp_task_allocation(suitability_matrix: List[List[float]]) -> List[Tuple[int, int]]:
    """
    Uses Integer Linear Programming (ILP) to maximize suitability-based task allocation.
    
    Parameters:
        suitability_matrix: A 2D list where the element at [i][j] represents the suitability score of robot i for task j.
    
    Returns:
        assignment: A list of (robot, task) pairs representing the final assignment.
    """
    num_robots = len(suitability_matrix)
    num_tasks = len(suitability_matrix[0])

    # Define the ILP problem
    problem = pulp.LpProblem("TaskAssignment", pulp.LpMaximize)

    # Define binary decision variables x_ij where x_ij = 1 if robot i is assigned to task j, else 0
    x = [[pulp.LpVariable(f"x_{i}_{j}", cat="Binary") for j in range(num_tasks)] for i in range(num_robots)]

    # Objective: Maximize total suitability score
    problem += pulp.lpSum(suitability_matrix[i][j] * x[i][j] for i in range(num_robots) for j in range(num_tasks))

    # Constraint: Each task can be assigned to at most one robot
    for j in range(num_tasks):
        problem += pulp.lpSum(x[i][j] for i in range(num_robots)) <= 1, f"Task_{j}_Assignment"

    # Constraint: Each robot can be assigned to at most one task
    for i in range(num_robots):
        problem += pulp.lpSum(x[i][j] for j in range(num_tasks)) <= 1, f"Robot_{i}_Capacity"

    # Solve the problem
    problem.solve(pulp.PULP_CBC_CMD(msg=False))

    # Collect the results
    assignment = []
    for i in range(num_robots):
        for j in range(num_tasks):
            if pulp.value(x[i][j]) == 1:
                assignment.append((i, j))

    return assignment

In [15]:
def print_robots(robots: List[CapabilityProfile]):
    for robot in robots:
        print(robot.robot_id)
        
def print_tasks(tasks: List[TaskDescription]):
    for task in tasks:
        print(task.task_id)

def calculate_suitability_matrix(robots: List[CapabilityProfile], tasks: List[TaskDescription], suitability_method: str) -> np.ndarray:
    """
    Calculates the suitability matrix for the given robots and tasks.
    
    Parameters:
        robots: List of robot profiles.
        tasks: List of task descriptions.
        suitability_method: The name of the suitability evaluation function.
    
    Returns:
        suitability_matrix: A 2D numpy array representing the suitability scores of each robot-task pair.
    """
    num_robots = len(robots)
    num_tasks = len(tasks)
    suitability_matrix = np.zeros((num_robots, num_tasks))

    # Evaluate suitability of each robot for each task
    for i, robot in enumerate(robots):
        for j, task in enumerate(tasks):
            suitability_score = globals()[suitability_method](robot, task)
            # NOTE: IMPORTANT can suitability_matrix hold robot.robot_id and task.task_id so that these values can be identified? maybe each score [i][j] can relate to a (robot, task) tuple that can be accessed like below?
            # suitability_matrix[i][j] = (suitability_score, (robot.robot_id, task.task_id)) # this way every suitability_score in the matrix is now associated with an unchanging pair and we don't have to interpret the indices
            suitability_matrix[i][j] = suitability_score
#             print(f"Suitability of {robot.robot_id} for {task.task_id}: {suitability_score:.2f}")

    # Print the suitability matrix
#     print("\nSuitability Matrix:")
    # print(suitability_matrix)
            
    return suitability_matrix

def assign_tasks_with_voting(robots: List[CapabilityProfile], tasks: List[TaskDescription], suitability_matrix: np.ndarray, num_candidates: int, voting_method: str):
    """
    Assigns tasks to robots using random assignment and ranks the assignments using the specified voting method.
    
    Parameters:
        robots: List of robot profiles.
        tasks: List of task descriptions.
        suitability_matrix: A 2D numpy array with suitability scores for each robot-task pair.
        num_candidates: Number of candidate assignments to generate.
        voting_method: The name of the voting function.
    
    Returns:
        (best_assignment, best_score, length): The best assignment, its suitability score, and the time taken for the voting process.
    """
    num_robots = len(robots)
    num_tasks = len(tasks)

    # Display generated robots and tasks
#     print("\nGenerated Robot Profiles:")
#     for robot in robots:
#         print(robot)

#     print("\nGenerated Task Descriptions:")
#     for task in tasks:
#         print(task)
    
#     random_assignments = generate_random_assignments(num_robots, num_tasks, num_candidates)
    random_assignments = generate_high_suitability_assignments(num_robots, num_tasks, suitability_matrix, num_candidates)
#     print("\nRandom Complete Assignments of Robots to Tasks:")
#     for i, assignment in enumerate(random_assignments):
#         print(f"Assignment {i+1}: {assignment}")
#         total_score = calculate_total_suitability(assignment[0], suitability_matrix)
#         print(f"Total Suitability Score for the Assignment: {total_score:.2f}")
    
    start = time.perf_counter_ns()
    # NOTE: are assignment rankings using robot_ids from the capability profiles?
    total_scores, assignment_ranking = globals()[voting_method](random_assignments, suitability_matrix)
    end = time.perf_counter_ns()
    length = (end - start) / 1000.0
    
#     print("\nTotal Suitability Scores for Each Assignment:")
#     for i, score in enumerate(total_scores):
#         print(f"Assignment {i+1}: {score:.2f}")

#     print("\nRanking of Assignments (from best to worst):")
#     for rank, assignment_index in enumerate(assignment_ranking):
#         print(f"Rank {rank+1}: Assignment {assignment_index+1}")

    best_ranking = 0
    while(check_zero_suitability(random_assignments[assignment_ranking[best_ranking]][0], suitability_matrix) and best_ranking < len(assignment_ranking)-1):
        best_ranking += 1
    if best_ranking == num_candidates-1:
        best_ranking = 0
    
#     print(best_ranking)
#     print(random_assignments[assignment_ranking[best_ranking]])
    # best_assignment = [[(robot, task), (robot, task), ...], [unassigned robots], [unassigned tasks]]
    best_assignment = random_assignments[assignment_ranking[best_ranking]]
    best_score = calculate_total_suitability(best_assignment[0], suitability_matrix)
    # print(f"BEST ASSIGNMENT: {best_assignment}")
    # print(f"BEST ASSIGNMENT[0]: {best_assignment[0]}")

    # returns the best assignment of robots to tasks and the total suitability score of the best assignment
    print(f"Score of the Best Assignment: {best_score:.2f}")
    print(f"Time taken for the voting process: {length:.2f} microseconds")
    return best_assignment, best_score, length

def assign_tasks_with_method(
    allocation_method: Callable[[List[List[float]]], List[Tuple[int, int]]],
    suitability_matrix: List[List[float]]
):
    """
    Assigns tasks using a specified allocation method and returns the allocation details.
    
    Parameters:
        allocation_method: The function used for task allocation (e.g., `cbba_task_allocation`, `ssia_task_allocation`, `ilp_task_allocation`).
        suitability_matrix: A 2D list where the element at [i][j] represents the suitability score of robot i for task j.
    
    Returns:
        (assignment, unassigned_robots, unassigned_tasks, total_score, allocation_time): A tuple containing:
      1. A list of assigned (robot, task) pairs.
      2. A list of unassigned robot indices.
      3. A list of unassigned task indices.
      4. The total suitability score of the assignment.
      5. The time taken for the allocation (in microseconds).
    """
    num_robots = len(suitability_matrix)
    num_tasks = len(suitability_matrix[0])
    
    # Start timing
    start_time = time.perf_counter_ns()
    
    # Get the assignment using the specified allocation method
    assignment = allocation_method(suitability_matrix)
    
    # End timing
    end_time = time.perf_counter_ns()
    allocation_time = (end_time - start_time) / 1000.0  # Convert nanoseconds to microseconds
    
    # Calculate total suitability score for the assignment
    total_score = calculate_total_suitability(assignment, suitability_matrix)
    
    # Determine unassigned robots and tasks
    assigned_robots = {robot for robot, _ in assignment}
    assigned_tasks = {task for _, task in assignment}
    
    unassigned_robots = [robot for robot in range(num_robots) if robot not in assigned_robots]
    unassigned_tasks = [task for task in range(num_tasks) if task not in assigned_tasks]

    print(f"Total Suitability Score: {total_score:.2f}")
    print(f"Time taken for allocation: {allocation_time:.2f} microseconds")
    
    return (assignment, unassigned_robots, unassigned_tasks), total_score, allocation_time

In [20]:
def simulate_time_step(
    robots: List[CapabilityProfile],
    tasks: List[TaskDescription],
    unassigned_robots: List[str],
    unassigned_tasks: List[str],
    suitability_method: str,
    occupied_locations: set,
    start_positions: dict,
    goal_positions: dict,
    time_step: float = 1.0,
    total_reward: float = 0.0,
    total_success: int = 0
) -> Tuple[int, float, int]:
    """
    Simulates a single time step, updating robot positions, task progress, and handling failures.

    Parameters:
        robots: List of all robots.
        tasks: List of all tasks.
        time_step: The time increment for the simulation step.
        total_reward: Accumulated reward from successfully completed tasks.

    Returns:
        (count, total_reward, total_success): A count of unassigned robots and the updated total reward.
    """
    count = 0  # Count of unassigned robots
    # Iterate through all robots to update their positions and tasks
    for robot in robots:
        if robot.assigned: # Check that robot is assigned
            if robot.current_task is not None: # Check that all assigned robots have a task
                # Get the assigned task
                task = robot.current_task

                # check if there is more path to traverse for the robot
                if robot.current_path and len(robot.current_path) > 1:

                    next_position = robot.current_path[1] # gives (x, y) coordinate of next step in path
                    occupied_locations.discard(robot.location) # remove current location from occupied set
                    robot.location = next_position # update location
                    # start position for this robot should be replaced, if not then must index by ID
                    start_positions[robot.robot_id] = next_position
                    occupied_locations.add(next_position) # update occupied set with robots new current location
                    robot.current_path.pop(0) # Move the robot one space by removing the first element from the robots path
                    robot.remaining_distance = len(robot.current_path) - 1 # recalculate the remaining distance and new location

                    if robot.remaining_distance <= 1:
                        # The robot has reached the task
                        robot.remaining_distance = 0
                        # this wont work here because once the robot reaches the task it will just keep resetting the time on task, I put it down below after completing the task
                        # robot.time_on_task = 0  # Reset time on task so it can begin work (time on task is a counter for how long it takes to complete a task)


                    if robot.battery_life == 0:  # Example 0.1% failure rate during navigation
                        # print(f"Robot {robot.robot_id} failed to reach task {task.task_id} due to mechanical failure.")
                        # get task id and task index
                        task_id = robot.current_task.task_id
                        t_index = [task.task_id for task in tasks ].index(task_id)
                        # unassign task
                        tasks[t_index].assigned_robot = None
                        tasks[t_index].assigned = False
                        robot.assigned = False

                        # move it to unassigned tasks list with check
                        tid = tasks[t_index].task_id
                        if tid not in unassigned_tasks:
                            unassigned_tasks.append(tid)
                        # unassign robot
                        robot.current_task = None
                        # move it to unassigned robots list with check
                        rid = robot.robot_id
                        if rid not in unassigned_robots:
                            unassigned_robots.append(rid)
                        continue

                # If the robot is at the task, increment time on task
                if robot.remaining_distance == 0:
                    #robot.time_on_task += time_step
                    robot.battery_life -= time_step
                    task.time_left -= time_step
                    # suitability = globals()[suitability_method](robot, task)
                    # failure_probability = 1 / (100 * (suitability + 1))  # Higher suitability, lower failure rate
                    # Check if the task is completed
                    if task.time_left <= 0:
                        # Mark task as completed
                        total_reward += task.reward
                        total_success += 1
                        robot.current_task = None
                        robot.tasks_successful += 1
                        robot.assigned = False
                        robot.time_on_task = 0  # Reset time on task so it can begin work (time on task is a counter for how long it takes to complete a task)
                        # move it to unassigned robots list with check
                        rid = robot.robot_id
                        if rid not in unassigned_robots:
                            unassigned_robots.append(rid)
                        print(f"Robot {robot.robot_id} completed task {task.task_id}")
                        tasks.remove(task)
                    elif robot.battery_life == 0:
                        task_id = robot.current_task.task_id
                        t_index = [task.task_id for task in tasks].index(task_id)
                        tasks[t_index].assigned_robot = None
                        tasks[t_index].assigned = False
                        if (task.reset_progress): # Only resets progress for certain tasks
                            task.time_left = task.time_to_complete
                        else:
                            task.time_to_complete = task.time_left
                        robot.current_task = None
                        robot.assigned = False

                        # move it to unassigned tasks list with check
                        tid = tasks[t_index].task_id
                        if tid not in unassigned_tasks:
                            unassigned_tasks.append(tid)
                        # move it to unassigned robots list with check
                        rid = robot.robot_id
                        if rid not in unassigned_robots:
                            unassigned_robots.append(rid)
                        continue
            else:
                print(f"Assigned robot {robot.robot_id} has no current task")
        else:
            count += 1
    return count, total_reward, total_success

def add_new_tasks(tasks: List[TaskDescription], unassigned_tasks: List[str], task_max_id: int, new_task_count: int, total_tasks: int, grid: List[List[int]], occupied_locations: set) -> Tuple[int, int]:
    """
    This function generates new unassigned tasks with random descriptions and adds them to the tasks list.
    It also updates the unassigned tasks list and occupied locations set.
    
    Parameters:
        tasks: List of existing task descriptions.
        unassigned_tasks: List of unassigned task IDs.
        task_max_id: The current maximum task ID to ensure unique IDs for new tasks.
        new_task_count: The number of new tasks to add.
        grid: The grid representing the environment where tasks are located.
        occupied_locations: A set of currently occupied locations to avoid conflicts.
        
    Returns:
        task_max_id: The updated maximum task ID after adding new tasks.
        total_tasks: The updated total number of tasks in the system.
    """
    for _ in range(new_task_count):
        task_id = f"T{task_max_id}"
        task_max_id += 1
        total_tasks += 1
        new_task = generate_random_task_description(task_id, grid, occupied_locations, tasks)
        tasks.append(new_task)
        unassigned_tasks.append(task_id)
    return task_max_id, total_tasks

def add_new_tasks_strict(tasks: List[TaskDescription], unassigned_tasks: List[str], task_max_id: int, new_task_count: int, total_tasks: int, grid: List[List[int]], occupied_locations: set) -> Tuple[int, int]:
    """
    This function generates new unassigned tasks with random descriptions and adds them to the tasks list.
    It also updates the unassigned tasks list and occupied locations set.
    
    Parameters:
        tasks: List of existing task descriptions.
        unassigned_tasks: List of unassigned task IDs.
        task_max_id: The current maximum task ID to ensure unique IDs for new tasks.
        new_task_count: The number of new tasks to add.
        grid: The grid representing the environment where tasks are located.
        occupied_locations: A set of currently occupied locations to avoid conflicts.
        
    Returns:
        task_max_id: The updated maximum task ID after adding new tasks.
        total_tasks: The updated total number of tasks in the system.
    """
    for _ in range(new_task_count):
        task_id = f"T{task_max_id}"
        task_max_id += 1
        total_tasks += 1
        new_task = generate_random_task_description_strict(task_id, grid, occupied_locations, tasks)
        tasks.append(new_task)
        unassigned_tasks.append(task_id)
    return task_max_id, total_tasks

def add_new_robots(robots: List[CapabilityProfile], unassigned_robots: List[str], robot_max_id: int, new_robot_count: int, grid: List[List[int]], occupied_locations: set) -> int:
    """
    This function generates new unassigned robots with random profiles and adds them to the robots list.
    It also updates the unassigned robots list and occupied locations set.
    
    Parameters:
        robots: List of existing robot profiles.
        unassigned_robots: List of unassigned robot IDs.
        robot_max_id: The current maximum robot ID to ensure unique IDs for new robots.
        new_robot_count: The number of new robots to add.
        grid: The grid representing the environment where robots operate.
        occupied_locations: A set of currently occupied locations to avoid conflicts.
        
    Returns:
        robot_max_id: The updated maximum robot ID after adding new robots.
        """
    for _ in range(new_robot_count):
        robot_id = f"R{robot_max_id}"
        robot_max_id += 1
        new_robot = generate_random_robot_profile(robot_id, grid, occupied_locations)
        robots.append(new_robot)
        unassigned_robots.append(robot_id)
        occupied_locations.add(new_robot.location)
        # NOTE: do not update start_positions until robots are given a task because they will be in a new location each time they are assigned a task if they get reassigned and CBS recalculates paths
        # NOTE: may need to make robot.current_path an empty list when unassigned to prevent unintentional movement
    return robot_max_id

def add_new_robots_strict(robots: List[CapabilityProfile], unassigned_robots: List[str], robot_max_id: int, new_robot_count: int, grid: List[List[int]], occupied_locations: set) -> int:
    """
    This function generates new unassigned robots with random profiles and adds them to the robots list.
    It also updates the unassigned robots list and occupied locations set.
    
    Parameters:
        robots: List of existing robot profiles.
        unassigned_robots: List of unassigned robot IDs.
        robot_max_id: The current maximum robot ID to ensure unique IDs for new robots.
        new_robot_count: The number of new robots to add.
        grid: The grid representing the environment where robots operate.
        occupied_locations: A set of currently occupied locations to avoid conflicts.
        
    Returns:
        robot_max_id: The updated maximum robot ID after adding new robots.
        """
    for _ in range(new_robot_count):
        robot_id = f"R{robot_max_id}"
        robot_max_id += 1
        new_robot = generate_random_robot_profile_strict(robot_id, grid, occupied_locations)
        robots.append(new_robot)
        unassigned_robots.append(robot_id)
        occupied_locations.add(new_robot.location)
        # NOTE: do not update start_positions until robots are given a task because they will be in a new location each time they are assigned a task if they get reassigned and CBS recalculates paths
        # NOTE: may need to make robot.current_path an empty list when unassigned to prevent unintentional movement
    return robot_max_id


def remove_random_robots(robots: List[CapabilityProfile], tasks: List[TaskDescription], unassigned_robots: List[str], unassigned_tasks: List[str], count: int, occupied_locations: set, start_positions: dict, goal_positions: dict):
    """
    This function selects a specified number of robots to remove from the robots list.
    It updates the unassigned robots and tasks lists, as well as the occupied locations set.
    
    Parameters:
        robots: List of existing robot profiles.
        tasks: List of existing task descriptions.
        unassigned_robots: List of unassigned robot IDs.
        unassigned_tasks: List of unassigned task IDs.
        count: The number of robots to remove.
        occupied_locations: A set of currently occupied locations to update.
        start_positions: A dictionary mapping robot IDs to their start positions.
        goal_positions: A dictionary mapping robot IDs to their goal positions.
        
    Returns:
        None
    """
    for _ in range(min(count, (len(robots) - 1))): # Make sure theres always at least one robot so CBS doesnt break
        robot_to_remove = random.choice(robots)
        occupied_locations.discard(robot_to_remove.location)
        if not robot_to_remove.assigned and robot_to_remove.current_task == None:
            if robot_to_remove.robot_id in unassigned_robots:
                # If the robot is unassigned, just remove it from the list
                unassigned_robots.remove(robot_to_remove.robot_id)
        else:
            task_id = robot_to_remove.current_task.task_id
            t_index = [task.task_id for task in tasks].index(task_id)
            tasks[t_index].assigned_robot = None
            tasks[t_index].assigned = False
            if task_id not in unassigned_tasks:
            # If the task is not already in the unassigned tasks list, add it
                unassigned_tasks.append(task_id)
            # in the case an assigned robot is removed we must update the start_positions and goal_positions lists to remove both locations so they are no longer used in the CBS pathfinding
            del start_positions[robot_to_remove.robot_id]
            del goal_positions[robot_to_remove.robot_id]
            # check if the robot is in the unassigned robots list and remove it
            if robot_to_remove.robot_id in unassigned_robots:
                # If the robot is unassigned, just remove it from the list
                unassigned_robots.remove(robot_to_remove.robot_id)
        robots.remove(robot_to_remove)
#         print(f"Robot {robot_to_remove.robot_id} left the system. It attempted {robot_to_remove.tasks_attempted} tasks and successfully completed {robot_to_remove.tasks_successful} of them.")

def reassign_robots_to_tasks(
        robots: List[CapabilityProfile], 
        tasks: List[TaskDescription], 
        num_candidates: int, 
        voting_method: str, 
        suitability_method: str, 
        unassigned_robots: List[str], 
        unassigned_tasks: List[str], 
        start_positions: dict, 
        goal_positions: dict,
        inertia_threshold: float = 0.1):
    """
    Reassigns unassigned robots to unassigned tasks using a voting method.
    Parameters:
        robots: List of all robot profiles.
        tasks: List of all task descriptions.
        num_candidates: Number of candidate assignments to generate.
        voting_method: The name of the voting function to use for ranking assignments.
        suitability_method: The name of the suitability evaluation function.
        unassigned_robots: List of unassigned robot IDs.
        unassigned_tasks: List of unassigned task IDs.
        start_positions: Dictionary mapping robot IDs to their start positions.
        goal_positions: Dictionary mapping robot IDs to their goal positions.
        Inertia threshold: minimum improvement in suitability required to steal an already‐assigned task.

    Returns:
        return_assignments: Dictionary mapping robot IDs to assigned task IDs.
        unassigned_robots: List of unassigned robot IDs after reassignment.
        unassigned_tasks: List of unassigned task IDs after reassignment.
        score: Total suitability score of the best assignment.
        length: Time taken for the voting process in microseconds.
    """
    urobots = [robot for robot in robots if not robot.assigned]
    utasks = [task for task in tasks if not task.assigned]

    suitability_matrix = calculate_suitability_matrix(urobots, utasks, suitability_method)
    output, score, length = assign_tasks_with_voting(
        urobots, utasks, suitability_matrix, num_candidates, voting_method)

    # this assigned_pairs only contains the unassigned robots and tasks, I may have to pass in the actual assigned_pairs to update it
    assigned_pairs = output[0] # list of tuples
    return_assignments = {}
    unassigned_robots = [urobots[i].robot_id for i in output[1]]
    unassigned_tasks = [utasks[j].task_id for j in output[2]]
    for (robot_idx, task_idx) in assigned_pairs:
        # NOTE: THIS IS RIGHT, indexes into the filtered list of assigned pairs with the unassigned robots and tasks
        # print(f"UROBOT: {urobots[robot_idx].robot_id} | UTASK: {utasks[task_idx].task_id}")
        r = urobots[robot_idx]
        t = utasks[task_idx]
        r.current_task = t
        r.tasks_attempted += 1
        t.assigned_robot = r
        r.assigned = True
        t.assigned = True
        # update start and goal positions when robots are assigned
        start_positions[r.robot_id] = r.location
        goal_positions[r.robot_id] = t.location
        return_assignments[r.robot_id] = t.task_id
        # return_assignments.update({urobots[robot_idx].robot_id : utasks[task_idx].task_id})

    # Check for stealing tasks from already assigned robots
    free_robots = [r for r in robots if not r.assigned]
    for task in tasks:
        current = task.assigned_robot
        if current is None:
            continue
        current_suitability = globals()[suitability_method](current, task)
        # find the best free robot for this task
        best, best_suit = None, current_suitability
        for r in free_robots:
            s = globals()[suitability_method](r, task)
            if s > best_suit:
                best, best_suit = r, s
        # Inertia check: if the best free robot's suitability is not significantly better, skip stealing
        if best and (best_suit - current_suitability) >= inertia_threshold:
            # unassign the current robot from the task
            current.current_task = None
            current.assigned = False
            if current.robot_id not in unassigned_robots:
                unassigned_robots.append(current.robot_id)
            # update the task's assigned robot
            best.current_task = task
            best.assigned = True
            best.tasks_attempted += 1
            task.assigned_robot = best
            start_positions[best.robot_id] = best.location
            goal_positions[best.robot_id] = task.location
            # remove from free list and unassigned robots
            free_robots.remove(best)
            if best.robot_id in unassigned_robots:
                unassigned_robots.remove(best.robot_id)
    
    print(f"Reassign Score: {score}, Reassign Length: {length}")

    return return_assignments, unassigned_robots, unassigned_tasks, score, length

def reassign_robots_to_tasks_with_method(
        robots: List[CapabilityProfile], 
        tasks: List[TaskDescription], 
        num_candidates: int, 
        voting_method: str, 
        suitability_method: str, 
        unassigned_robots: List[str], 
        unassigned_tasks: List[str], 
        allocation_method: Callable[[List[List[float]]], List[Tuple[int, int]]], 
        start_positions: dict, 
        goal_positions: dict,
        inertia_threshold: float = 0.1):
    """
    Reassigns unassigned robots to unassigned tasks using a specified allocation method.
    Parameters:
        robots: List of all robot profiles.
        tasks: List of all task descriptions.
        num_candidates: Number of candidate assignments to generate.
        voting_method: The name of the voting function to use for ranking assignments.
        suitability_method: The name of the suitability evaluation function.
        unassigned_robots: List of unassigned robot IDs.
        unassigned_tasks: List of unassigned task IDs.
        allocation_method: The function used for task allocation (e.g., `cbba_task_allocation`, `ssia_task_allocation`, `ilp_task_allocation`).
        start_positions: Dictionary mapping robot IDs to their start positions.
        goal_positions: Dictionary mapping robot IDs to their goal positions.
        Inertia threshold: minimum improvement in suitability required to steal an already‐assigned task.
        
    Returns:
        return_assignments: Dictionary mapping robot IDs to assigned task IDs.
        unassigned_robots: List of unassigned robot IDs after reassignment.
        unassigned_tasks: List of unassigned task IDs after reassignment.
        score: Total suitability score of the best assignment.
        length: Time taken for the voting process in microseconds.
    """
    urobots = [robot for robot in robots if not robot.assigned]
    utasks  = [task for task in tasks if not task.assigned]

    suitability_matrix = calculate_suitability_matrix(urobots, utasks, suitability_method)
    output, score, length = assign_tasks_with_method(allocation_method, suitability_matrix)

    assigned_pairs = output[0]
    return_assignments = {}
    unassigned_robots = [urobots[i].robot_id for i in output[1]]
    unassigned_tasks = [utasks[j].task_id for j in output[2]]
    for (robot_idx, task_idx) in assigned_pairs:
        # print(f"UROBOT: {urobots[robot_idx].robot_id} | UTASK: {utasks[task_idx].task_id}")
        r = urobots[robot_idx]
        t = utasks[task_idx]
        r.current_task = t
        r.tasks_attempted += 1
        t.assigned_robot = r
        r.assigned = True
        t.assigned = True
        # update start and goal positions when robots are assigned
        start_positions[r.robot_id] = r.location
        goal_positions[r.robot_id] = t.location
        return_assignments[r.robot_id] = t.task_id
    
    # Check for stealing tasks from already assigned robots
    free_robots = [r for r in robots if not r.assigned]
    for task in tasks:
        current = task.assigned_robot
        if current is None:
            continue
        current_suitability = globals()[suitability_method](current, task)
        # find the best free robot for this task
        best, best_suit = None, current_suitability
        for r in free_robots:
            s = globals()[suitability_method](r, task)
            if s > best_suit:
                best, best_suit = r, s
        # Inertia check: if the best free robot's suitability is not significantly better, skip stealing
        if best and (best_suit - current_suitability) >= inertia_threshold:
            # unassign the current robot from the task
            current.current_task = None
            current.assigned = False
            if current.robot_id not in unassigned_robots:
                unassigned_robots.append(current.robot_id)
            # update the task's assigned robot
            best.current_task = task
            best.assigned = True
            best.tasks_attempted += 1
            task.assigned_robot = best
            start_positions[best.robot_id] = best.location
            goal_positions[best.robot_id] = task.location
            # remove from free list and unassigned robots
            free_robots.remove(best)
            if best.robot_id in unassigned_robots:
                unassigned_robots.remove(best.robot_id)

    print(f"Reassign Score: {score}, Reassign Length: {length}")

    return return_assignments, unassigned_robots, unassigned_tasks, score, length
    
def create_obstacle_list(grid):
    """Create a reusable obstacle list from the grid for fast lookups"""
    obstacle_list = []
    for r in range(len(grid)):
        for c in range(len(grid[r])):
            if grid[r][c] == 1:
                obstacle_list.append((r, c)) # store all obstacles in a list
    return obstacle_list

def build_cbs_agents(robots, start_positions, goal_positions):
    """
    Return a fresh list of {'name','start','goal'} dicts for CBS,
    based on whatever robots are currently marked assigned.
    """
    return [
        {
            'name': r.robot_id,
            'start': start_positions[r.robot_id],
            'goal':  goal_positions[r.robot_id],
        }
        for r in robots
        if r.assigned and r.current_task is not None
    ]

In [ ]:
# import multiprocessing
import time
import psutil
# CBS block

def main_simulation(num_robots: int, num_tasks: int, num_candidates: int, voting_method: str, suitability_method: str, max_time_steps: int, add_tasks: bool, add_robots: bool, remove_robots: bool, tasks_to_add: int = 1, robots_to_add: int = 1, robots_to_remove: int = 1):
    # Load the MAPF benchmark map
    map_file = r"test_small_open.map"
    #input_file = r"input.txt"
    # output_file = r"C:\Users\owner\Documents\PhD\TierLab\VBTA - Original Commented\output.txt"
    grid = load_map(map_file) # 2D list of 0/1 representing the map
    # TODO: create reuseable map dictionary from the grid so we can pass it to the CBS planner 
    # So take in the map file, and create a map dictionary with the dimensions and obstacles, 
    # then for the agents, as they are created and assigned we can update the start and goal positions
    # but unassigned robots will not be a part of the agents list until they are assigned a task because 
    # they will not be moving because they do not have a goal to reach
    dims = (len(grid), len(grid[0])) # dimensions of the map grid
    obstacles = create_obstacle_list(grid) # list of obstacle coordinates
    # Create the map dictionary to pass to the CBS planner
    map_dict = {
        'dimension': dims,
        'obstacles': obstacles
    }

    # print(f"MAP DICTIONARY: {map_dict}")
    print(f"SUITABILITY METHOD: {suitability_method}")

    # get initial positions list to reuse later
    initial_positions = set()

    # generate_random_robot_profile now takes in the map grid and occupied positions and passes them to get_random_free_position function
    robots = [generate_random_robot_profile(f"R{i+1}", grid, initial_positions) for i in range(num_robots)]
    # changed how tasks are made here so there are no overlapping tasks
    tasks = [generate_random_task_description(f"T{i+1}", grid, initial_positions, []) for i in range(num_tasks)]
    initial_robots = copy.deepcopy(robots)
    initial_tasks = copy.deepcopy(tasks)
    robot_max_id = len(robots)+1
    task_max_id = len(tasks)+1
    total_reward = 0.0
    total_success = 0.0
    total_tasks = num_tasks
    total_reassignment_time = 0.0
    total_reassignment_score = 0.0
    total_reassignments = 0

    
    # then propogate the initial occupied positions
    # for r in robots:
    #     row = int(r.location[0][0])
    #     col = int(r.location[0][1])
    #     initial_positions.add((row, col))
    for r in robots:
        # add the robot's initial position to the occupied positions set
        initial_positions.add(r.location)

    # copy the initial positions set to use over and over
    # NOTE: occupied_positions does not need to be added to the list of obstacles because CBS will have agents avoid collisions
    occupied_positions = set(initial_positions) # use the occcupied positions as the current positions for CBS, this is just as a occupation check, not a start position

    # Calculate suitability matrix
    suitability_matrix = calculate_suitability_matrix(robots, tasks, suitability_method)
    
    # Perform task assignment using voting
    output, score, length = assign_tasks_with_voting(
        robots, tasks, suitability_matrix, num_candidates, voting_method)
    
    assigned_pairs = output[0]  # list of tuples (robot_idx, task_idx)
    for robot_idx, task_idx in assigned_pairs:
        r = robots[robot_idx]
        t = tasks[task_idx]

        r.current_task = t
        r.assigned = True
        r.tasks_attempted = 1

        t.assigned_robot = r
        t.assigned = True
    
    # rebuild all assignment dictionaries from the robot and task flags
    assigned_robots = {r.robot_id: r.current_task.task_id for r in
                       robots if r.assigned and r.current_task is not None}
    unassigned_tasks = [t.task_id for t in tasks if not t.assigned]
    unassigned_robots = [r.robot_id for r in robots if not r.assigned]

    start_positions = {
        r.robot_id: r.location 
        for r in robots
        if r.assigned and r.current_task is not None
    }
    goal_positions = {
        r.robot_id: r.current_task.location
        for r in robots
        if r.assigned and r.current_task is not None
    }

    
    # assigned_pairs = output[0] # list of [(robot, task), (robot, task), ...] assignments
    # unassigned_robots = [robots[val].robot_id for val in output[1]]
    # unassigned_tasks = [tasks[val].task_id for val in output[2]]
    # assigned_robots = {}
    # for (robot_idx, task_idx) in assigned_pairs:
    #     assigned_robots[robots[robot_idx].robot_id] = tasks[task_idx].task_id

    
    print(f"ROBOTS: {[rob.robot_id for rob in robots]}")
    print(f"TASKS: {[tas.task_id for tas in tasks]}")
    # print(f"ASSIGNED PAIRS: {assigned_pairs}")
    print(f"ASSIGNED ROBOTS: {assigned_robots}")

    # we also run the other assignment methods here to compare their outputs and scores later
    cbba_output, cbba_score, cbba_length = assign_tasks_with_method(cbba_task_allocation,suitability_matrix)
    ssia_output, ssia_score, ssia_length = assign_tasks_with_method(ssia_task_allocation,suitability_matrix)
    ilp_output, ilp_score, ilp_length = assign_tasks_with_method(ilp_task_allocation,suitability_matrix)
    
    # start_positions = {}
    # goal_positions = {}

    # for robot in robots:
    #     start_positions[robot.robot_id] = robot.location[0]

    # for r_id, t_id in assigned_pairs:
    #     print(f"ROBOT: {r_id}")
    #     print(f"TASK: {t_id}")
    #     # find the corresponding index for the task and robot
    #     t_index = [task.task_id for task in tasks].index(t_id)
    #     r_index = [robot.robot_id for robot in robots].index(r_id)
    #     print(f"ROBOT INDEX: {r_index}")
    #     print(f"TASK INDEX: {t_index}")
    #     # assign the current task to robot
    #     robots[r_index].current_task = tasks[t_index]
    #     # assign tasks attempted value
    #     robots[r_index].tasks_attempted = 1
    #     # assign current robot to task
    #     tasks[t_index].assigned_robot = robots[r_index]
    #     # assign the goal position with the AGENTS STRING ID not the index
    #     goal_positions[r_id] = tasks[t_index].location[0]
    #     robots[r_index].assigned = True

    # TODO: create an "agents" dictionary to pass to CBS with the start and goal positions and the agents string IDs
    # agents = []
    # for robot in robots:
    #     if robot.assigned and robot.current_task is not None:
    #         agents.append({
    #             'name': robot.robot_id,
    #             'start': start_positions[robot.robot_id],
    #             'goal': goal_positions[robot.robot_id]
    #         })
    agents = build_cbs_agents(robots, start_positions, goal_positions)

    print(f"AGENTS LIST: {agents}")

    # Create the input data dictionary for CBS, this will be passed to the CBS planner
    input_data = {
        'map' : {
            'dimension': map_dict['dimension'],
            'obstacles': map_dict['obstacles']
        },
        'agents': agents,
    }

    # print(f"INPUT DATA: {input_data}")

    # TODO: finish implementing the new CBS method into the simulation
    # NOTE: this will not work with the current setup, we need to pass in the
    # start and goal positions as dictionaries and a dictionary with a map representation
    env = Environment(
        dimension=map_dict['dimension'],
        agents=input_data['agents'],
        obstacles=map_dict['obstacles'],
    )
    planner = CBS(env)
    solution, nodes_expanded, conflicts = planner.search()

    print(f"SOLUTION: {solution}")

    if solution is None:
        print("CBS could not find a conflict free path assignment for all agents")
        # could possibly fall back on the simple method here if we get a lot of issues
    else:

        # Iterate through the agents and their schedules
        for agent, schedule in solution.items():
            print(f"Schedule for {agent}:")

            # Iterate through the schedule (list of dictionaries)
            for point in schedule:
                # Access each entry in the schedule
                # print(f"  t: {point['t']}, x: {point['x']}, y: {point['y']}")
                # Assign the paths to the robots
                rid = [robot.robot_id for robot in robots].index(agent)
                robots[rid].current_path.append((point['x'], point['y']))
            print(f"Robot {agent} path: {robots[rid].current_path}")

    # Keep track of previous state to not run CBS when there are no changes
    prev_state = {
        "robot_ids": set(robot.robot_id for robot in robots),
        "assigned": {robot.robot_id for robot in robots if robot.assigned},
        "unassigned": set(unassigned_robots)
    }
    
    # End the simulation if nothing changes for 3 timesteps (hopefully the tasks are done and the robots arent moving)
    time_steps_unchanged = 0

    for time_step in range(max_time_steps):
        # print(f"\n--- Time Step {time_step + 1} ---")
        # print(f"OCCUPIED POSITIONS: {occupied_positions}")
        print(f"START POSITIONS: {start_positions}")
        print(f"GOAL POSITONS: {goal_positions}")
        # print(f"UNASSIGNED ROBOTS: {unassigned_robots}")
        # print(f"UNASSIGNED TASKS: {unassigned_tasks}")
        # # print(f"ASSIGNED ROBOTS: {[rob.assigned for rob in robots]}")
        # print(f"ALL ROBOTS: {len(robots)}")
        # print(f"ALL TASKS: {len(tasks)}")
        # print(f"LIST OF ALL ROBOTS: {[rob.robot_id for rob in robots]}")
        # print(f"LIST OF ALL TASKS: {[tas.task_id for tas in tasks]}")

        # before each time step, refresh the unassigned robots and tasks lists
        unassigned_robots = [r.robot_id for r in robots if not r.assigned]
        unassigned_tasks = [t.task_id for t in tasks if not t.assigned]

        # Simulate time step
        unassigned_count, total_reward, total_success = simulate_time_step(robots, tasks, unassigned_robots, unassigned_tasks, suitability_method, occupied_positions, start_positions, goal_positions, 1.0, total_reward, total_success)
        # Periodically add new tasks and robots
        if add_tasks and time_step + 1 <= 2 and random.random() < 0.5: # add tasks only in the first 2 time steps, and randomly
            print(f"ADDING NEW TASKS AT TIME STEP {time_step + 1}")
            task_max_id, total_tasks = add_new_tasks(tasks, unassigned_tasks, task_max_id, random.randint(0, tasks_to_add), total_tasks, grid, occupied_positions)
        if add_robots and time_step + 1 <= 4 and random.random() < 0.5: # add robots only in the first 4 time steps, and randomly
            print(f"ADDING NEW ROBOTS AT TIME STEP {time_step + 1}")
            robot_max_id = add_new_robots(robots, unassigned_robots, robot_max_id, random.randint(0, robots_to_add), grid, occupied_positions)

        # Periodically remove robots
        if remove_robots and time_step + 1 <= 4 and random.random() < 0.5: # remove robots only in the first 4 time steps, and randomly
            if len(assigned_robots) > 1: # Otherwise will break CBS, we need at least one agent for things to run smoothly
                print(f"REMOVING RANDOM ROBOTS AT TIME STEP {time_step + 1}")
                remove_random_robots(robots, tasks, unassigned_robots, unassigned_tasks, random.randint(0, robots_to_remove), occupied_positions, start_positions, goal_positions)
            

        # Reassign unassigned robots to unassigned tasks
        # TODO: change this to make sure newly assigned robot is more suitable for the task
        # so we will run it when there are unassigned robots AND a state change, otherwise theres no reason to keep checking
        if unassigned_robots and unassigned_tasks:
            total_reassignments += 1
            _, unassigned_robots, unassigned_tasks, reassign_score, reassign_length = reassign_robots_to_tasks(
                robots, tasks, num_candidates, voting_method, suitability_method, 
                unassigned_robots, unassigned_tasks, start_positions, goal_positions)
            total_reassignment_time += reassign_length
            total_reassignment_score += reassign_score
            # regenerate the master assigned robots dictionary
            assigned_robots = {
                r.robot_id: r.current_task.task_id 
                for r in robots 
                if r.assigned and r.current_task is not None
            }

            print(f"ROBOTS REASSIGNED")
            # print(f"NEW ASSIGNMENTS FROM REASSIGNMENT: {new_assignments}")
            print(f"ASSIGNED ROBOTS: {assigned_robots}")
            print(f"ALL ROBOTS: {[robot.robot_id for robot in robots]}")
            
        
        # Update start and goal positions before cbs
        for robot in robots:
            if robot.assigned and robot.current_task:
                start_positions[robot.robot_id] = robot.location
                goal_positions[robot.robot_id] = robot.current_task.location
            else:
                start_positions.pop(robot.robot_id, None)
                goal_positions.pop(robot.robot_id, None)

        current_state = {
            "robot_ids": set(robot.robot_id for robot in robots),
            "assigned": {robot.robot_id for robot in robots if robot.assigned},
            "unassigned": set(unassigned_robots)
        }
        
        if current_state != prev_state and [rob.assigned for rob in robots].count(True) >= 1:
            print("State change, re-run CBS...")
            print(f"AMOUNT OF ASSIGNED ROBOTS: {len([rob.current_task for rob in robots])}")
            print(f"ASSIGNED ROBOTS: {[rob.assigned for rob in robots].count(True)}")
            # NOTE: use occupied positions as current positions for running CBS, update goal positions whenever needed
            print(f"OCCUPIED POSITIONS: {occupied_positions}")
            print(f"START POSITIONS: {start_positions}")
            print(f"GOAL POSITONS: {goal_positions}")
            print(f"UNASSIGNED ROBOTS: {unassigned_robots}")
            print(f"UNASSIGNED TASKS: {unassigned_tasks}")
            print(f"ALL ROBOTS: {len(robots)}")
            print(f"ALL TASKS: {len(tasks)}")
            print(f"LIST OF ALL ROBOTS: {[rob.robot_id for rob in robots]}")
            print(f"LIST OF ALL TASKS: {[tas.task_id for tas in tasks]}")
            agents = build_cbs_agents(robots, start_positions, goal_positions)

            print(f"AGENTS LIST: {agents}")
            
            # Validation block
            start_locations = [agent['start'] for agent in agents]
            if len(start_locations) != len(set(start_locations)):
                print("ERROR: Duplicate start locations found in agent list. Aborting CBS.")
                # You can decide how to handle this. For now, we can skip the planning step for this cycle.
                # To prevent a crash, you could set solution to None and handle it.
                solution = None
            else:
                # instantiate environment and planner
                env = Environment(
                    dimension=map_dict['dimension'],
                    agents=agents,
                    obstacles=map_dict['obstacles']
                )
                planner = CBS(env)
                solution, nodes_expanded, conflicts = planner.search()
                print(f"CBS COMPLETE. New solution: {solution}")

            if solution:
                # update each robot's path based on the new CBS solution
                for robot_id, schedule in solution.items():
                    ridx = [robot.robot_id for robot in robots].index(robot_id)
                    for point in schedule:
                        # print(f"  t: {point['t']}, x: {point['x']}, y: {point['y']}")
                        # Assign the paths to the robots
                        robots[ridx].current_path.append((point['x'], point['y']))
                    # ridx = [r.robot_id for r in robots].index(robot_id)
                    # robots[ridx].current_path = schedule
                    robots[ridx].remaining_distance = len(schedule) - 1
                prev_state = current_state.copy()

            else:
                print("CBS could not find a conflict free path assignment for all agents")
                # could possibly fall back on the simple method here if we get a lot of issues
                # but for now, we will just skip CBS and continue with the simulation
                print("Skipping CBS...")
                time_steps_unchanged += 1
                if time_steps_unchanged >= 3:
                    print("No state change for 3 time steps, ending simulation.")
                    break
        else:
            print("No state change, skip CBS...")
            print(f"ALL ROBOTS: {len(robots)}")
            print(f"ALL TASKS: {len(tasks)}")
            print(f"LIST OF ALL ROBOTS: {[rob.robot_id for rob in robots]}")
            print(f"LIST OF ALL TASKS: {[tas.task_id for tas in tasks]}")
            # if time_steps_unchanged < 3:
            #     time_steps_unchanged += 1
            #     print("No state change, skip CBS...")
            #     print(f"ALL ROBOTS: {len(robots)}")
            #     print(f"ALL TASKS: {len(tasks)}")
            #     print(f"LIST OF ALL ROBOTS: {[rob.robot_id for rob in robots]}")
            #     print(f"LIST OF ALL TASKS: {[tas.task_id for tas in tasks]}")
            # else:
            #     break

    overall_success_rate = total_success / total_tasks
    print(f"Voting: Total reward: {total_reward}, Overall success rate: {overall_success_rate:.2%}, Tasks completed: {total_success}, Reassignment Time: {total_reassignment_time}, Reassignment Score: {total_reassignment_score}, total reassignments: {total_reassignments}")
#     for robot in robots:
#         print(f"Robot {robot.robot_id} attempted {robot.tasks_attempted} tasks and successfully completed {robot.tasks_successful} of them.")

    robots = copy.deepcopy(initial_robots)
    tasks = copy.deepcopy(initial_tasks)
    robot_max_id = len(robots)+1
    task_max_id = len(tasks)+1
    total_reward = 0.0
    total_success = 0.0
    total_tasks = num_tasks
    total_reassignment_time = 0.0
    total_reassignment_score = 0.0
    total_reassignments = 0

    # propogate the initial occupied positions again
    initial_positions = set()
    # for r in robots:
    #     row = int(r.location[0][0])
    #     col = int(r.location[0][1])
    #     initial_positions.add((row, col))
    for r in robots:
        # add the robot's initial position to the occupied positions set
        initial_positions.add(r.location)

    occupied_positions = set(initial_positions)
    
    assigned_pairs = cbba_output[0]
    for robot_idx, task_idx in assigned_pairs:
        r = robots[robot_idx]
        t = tasks[task_idx]

        r.current_task = t
        r.assigned = True
        r.tasks_attempted = 1

        t.assigned_robot = r
        t.assigned = True
        
    # rebuild all assignment dictionaries from the robot and task flags
    assigned_robots = {r.robot_id: r.current_task.task_id for r in
                       robots if r.assigned and r.current_task is not None}
    unassigned_tasks = [t.task_id for t in tasks if not t.assigned]
    unassigned_robots = [r.robot_id for r in robots if not r.assigned]

    start_positions = {
        r.robot_id: r.location
        for r in robots
        if r.assigned and r.current_task is not None
    }

    goal_positions = {
        r.robot_id: r.current_task.location
        for r in robots
        if r.assigned and r.current_task is not None
    }

    print(f"ROBOTS: {[rob.robot_id for rob in robots]}")
    print(f"TASKS: {[tas.task_id for tas in tasks]}")
    print(f"ASSIGNED ROBOTS: {assigned_robots}")

    agents = build_cbs_agents(robots, start_positions, goal_positions)

    input_data = {
        'map' : {
            'dimension': map_dict['dimension'],
            'obstacles': map_dict['obstacles']
        },
        'agents': agents,
    }

    env = Environment(
        dimension=map_dict['dimension'],
        agents=input_data['agents'],
        obstacles=map_dict['obstacles'],
    )
    planner = CBS(env)
    solution, nodes_expanded, conflicts = planner.search()

    print(f"SOLUTION: {solution}")

    if solution is None:
        print("CBS could not find a conflict free path assignment for all agents")
    
    else:
        for agent, schedule in solution.items():
            print(f"Schedule for {agent}:")

            for point in schedule:
                rid = [robot.robot_id for robot in robots].index(agent)
                robots[rid].current_path.append((point['x'], point['y']))
            print(f"Robot {agent} path: {robots[rid].current_path}")

    prev_state = {
        "robot_ids": set(robot.robot_id for robot in robots),
        "assigned": {robot.robot_id for robot in robots if robot.assigned},
        "unassigned": set(unassigned_robots)
    }
    time_steps_unchanged = 0
    
    for time_step in range(max_time_steps):
        # print(f"\n--- Time Step {time_step + 1} ---")
        # print(f"OCCUPIED POSITIONS: {occupied_positions}")
        # print(f"START POSITIONS: {start_positions}")
        # print(f"GOAL POSITONS: {goal_positions}")
        # print(f"UNASSIGNED ROBOTS: {unassigned_robots}")
        # print(f"UNASSIGNED TASKS: {unassigned_tasks}")
        # # print(f"ASSIGNED ROBOTS: {[rob.assigned for rob in robots]}")
        # print(f"ALL ROBOTS: {len(robots)}")
        # print(f"ALL TASKS: {len(tasks)}")
        # print(f"LIST OF ALL ROBOTS: {[rob.robot_id for rob in robots]}")
        # print(f"LIST OF ALL TASKS: {[tas.task_id for tas in tasks]}")

        unassigned_robots = [r.robot_id for r in robots if not r.assigned]
        unassigned_tasks = [t.task_id for t in tasks if not t.assigned]

        unassigned_count, total_reward, total_success = simulate_time_step(robots, tasks, unassigned_robots, unassigned_tasks, suitability_method, occupied_positions, start_positions, goal_positions, 1.0, total_reward, total_success)
        if add_tasks and time_step + 1 <= 2 and random.random() < 0.5: # add tasks only in the first 2 time steps, and randomly
            print(f"ADDING NEW TASKS AT TIME STEP {time_step + 1}")
            task_max_id, total_tasks = add_new_tasks(tasks, unassigned_tasks, task_max_id, random.randint(0, tasks_to_add), total_tasks, grid, occupied_positions)
        if add_robots and time_step + 1 <= 4 and random.random() < 0.5: # add robots only in the first 4 time steps, and randomly
            print(f"ADDING NEW ROBOTS AT TIME STEP {time_step + 1}")
            robot_max_id = add_new_robots(robots, unassigned_robots, robot_max_id, random.randint(0, robots_to_add), grid, occupied_positions)

        if remove_robots and time_step + 1 <= 4 and random.random() < 0.5: # remove robots only in the first 4 time steps, and randomly
            if len(assigned_robots) > 1:
                print(f"REMOVING RANDOM ROBOTS AT TIME STEP {time_step + 1}")
                remove_random_robots(robots, tasks, unassigned_robots, unassigned_tasks, random.randint(0, robots_to_remove), occupied_positions, start_positions, goal_positions)

        if unassigned_robots and unassigned_tasks:
            total_reassignments += 1
            _, unassigned_robots, unassigned_tasks, reassign_score, reassign_length = reassign_robots_to_tasks_with_method(
                robots, tasks, num_candidates, voting_method, suitability_method, 
                unassigned_robots, unassigned_tasks, cbba_task_allocation, start_positions, goal_positions)
            total_reassignment_time += reassign_length
            total_reassignment_score += reassign_score

            # regenerate the master assigned robots dictionary based on flags
            assigned_robots = {
                r.robot_id: r.current_task.task_id 
                for r in robots 
                if r.assigned and r.current_task is not None
            }
            print(f"ROBOTS REASSIGNED")
            print(f"ASSIGNED ROBOTS: {assigned_robots}")
            print(f"ALL ROBOTS: {[robot.robot_id for robot in robots]}")

        for robot in robots:
            if robot.assigned and robot.current_task is not None:
                start_positions[robot.robot_id] = robot.location
                goal_positions[robot.robot_id] = robot.current_task.location
            else:
                start_positions.pop(robot.robot_id, None)
                goal_positions.pop(robot.robot_id, None)
        
        current_state = {
            "robot_ids": set(robot.robot_id for robot in robots),
            "assigned": {robot.robot_id for robot in robots if robot.assigned},
            "unassigned": set(unassigned_robots)
        }

        if current_state != prev_state and [rob.assigned for rob in robots].count(True) >= 1:
            print("State change, re-run CBS...")
            print(f"AMOUNT OF ASSIGNED ROBOTS: {len([rob.current_task for rob in robots])}")
            print(f"ASSIGNED ROBOTS: {[rob.assigned for rob in robots].count(True)}")
            print(f"OCCUPIED POSITIONS: {occupied_positions}")
            print(f"START POSITIONS: {start_positions}")
            print(f"GOAL POSITONS: {goal_positions}")
            print(f"UNASSIGNED ROBOTS: {unassigned_robots}")
            print(f"UNASSIGNED TASKS: {unassigned_tasks}")
            print(f"ALL ROBOTS: {len(robots)}")
            print(f"ALL TASKS: {len(tasks)}")
            print(f"LIST OF ALL ROBOTS: {[rob.robot_id for rob in robots]}")
            print(f"LIST OF ALL TASKS: {[tas.task_id for tas in tasks]}")

            agents = build_cbs_agents(robots, start_positions, goal_positions)

            # Validation block
            start_locations = [agent['start'] for agent in agents]
            if len(start_locations) != len(set(start_locations)):
                print("ERROR: Duplicate start locations found in agent list. Aborting CBS.")
                # You can decide how to handle this. For now, we can skip the planning step for this cycle.
                # To prevent a crash, you could set solution to None and handle it.
                solution = None
            else:
                # instantiate environment and planner
                env = Environment(
                    dimension=map_dict['dimension'],
                    agents=agents,
                    obstacles=map_dict['obstacles']
                )
                planner = CBS(env)
                solution, nodes_expanded, conflicts = planner.search()
                print(f"CBS COMPLETE. New solution: {solution}")

            if solution:
                # update each robot's path based on the new CBS solution
                for robot_id, schedule in solution.items():
                    ridx = [robot.robot_id for robot in robots].index(robot_id)
                    for point in schedule:
                        # print(f"  t: {point['t']}, x: {point['x']}, y: {point['y']}")
                        # Assign the paths to the robots
                        robots[ridx].current_path.append((point['x'], point['y']))
                    # ridx = [r.robot_id for r in robots].index(robot_id)
                    # robots[ridx].current_path = schedule
                    robots[ridx].remaining_distance = len(schedule) - 1
                prev_state = current_state.copy()

            else:
                print("CBS could not find a conflict free path assignment for all agents")
                # could possibly fall back on the simple method here if we get a lot of issues
                # but for now, we will just skip CBS and continue with the simulation
                print("Skipping CBS...")
                time_steps_unchanged += 1
                if time_steps_unchanged >= 3:
                    print("No state change for 3 time steps, ending simulation.")
                    break
        else:
            print("No state change, skip CBS...")
            print(f"ALL ROBOTS: {len(robots)}")
            print(f"ALL TASKS: {len(tasks)}")
            print(f"LIST OF ALL ROBOTS: {[rob.robot_id for rob in robots]}")
            print(f"LIST OF ALL TASKS: {[tas.task_id for tas in tasks]}")

    overall_success_rate = total_success / total_tasks
    print(f"CBBA: Total reward: {total_reward}, Overall success rate: {overall_success_rate:.2%}, Tasks completed: {total_success}, Reassignment Time: {total_reassignment_time}, Reassignment Score: {total_reassignment_score}, total reassignments: {total_reassignments}")
# #     for robot in robots:
# #         print(f"Robot {robot.robot_id} attempted {robot.tasks_attempted} tasks and successfully completed {robot.tasks_successful} of them.")

    robots = copy.deepcopy(initial_robots)
    tasks = copy.deepcopy(initial_tasks)
    robot_max_id = len(robots)+1
    task_max_id = len(tasks)+1
    total_reward = 0.0
    total_success = 0.0
    total_tasks = num_tasks
    total_reassignment_time = 0.0
    total_reassignment_score = 0.0
    total_reassignments = 0

    # propogate the initial occupied positions again
    initial_positions = set()
    # for r in robots:
    #     row = int(r.location[0][0])
    #     col = int(r.location[0][1])
    #     initial_positions.add((row, col))
    for r in robots:
        # add the robot's initial position to the occupied positions set
        initial_positions.add(r.location)

    occupied_positions = set(initial_positions)

    assigned_pairs = ssia_output[0]
    for robot_idx, task_idx in assigned_pairs:
        r = robots[robot_idx]
        t = tasks[task_idx]

        r.current_task = t
        r.assigned = True
        r.tasks_attempted = 1

        t.assigned_robot = r
        t.assigned = True

    # rebuild all assignment dictionaries from the robot and task flags
    assigned_robots = {r.robot_id: r.current_task.task_id for r in
                          robots if r.assigned and r.current_task is not None}
    unassigned_tasks = [t.task_id for t in tasks if not t.assigned]
    unassigned_robots = [r.robot_id for r in robots if not r.assigned]

    start_positions = {
        r.robot_id: r.location
        for r in robots
        if r.assigned and r.current_task is not None
    }

    goal_positions = {
        r.robot_id: r.current_task.location
        for r in robots
        if r.assigned and r.current_task is not None
    }

    print(f"ROBOTS: {[rob.robot_id for rob in robots]}")
    print(f"TASKS: {[tas.task_id for tas in tasks]}")
    print(f"ASSIGNED ROBOTS: {assigned_robots}")

    agents = build_cbs_agents(robots, start_positions, goal_positions)

    input_data = {
        'map' : {
            'dimension': map_dict['dimension'],
            'obstacles': map_dict['obstacles']
        },
        'agents': agents,
    }

    env = Environment(
        dimension=map_dict['dimension'],
        agents=input_data['agents'],
        obstacles=map_dict['obstacles'],
    )
    planner = CBS(env)
    solution, nodes_expanded, conflicts = planner.search()

    print(f"SOLUTION: {solution}")

    if solution is None:
        print("CBS could not find a conflict free path assignment for all agents")
    
    else:
        for agent, schedule in solution.items():
            print(f"Schedule for {agent}:")

            for point in schedule:
                rid = [robot.robot_id for robot in robots].index(agent)
                robots[rid].current_path.append((point['x'], point['y']))
            print(f"Robot {agent} path: {robots[rid].current_path}")

    prev_state = {
        "robot_ids": set(robot.robot_id for robot in robots),
        "assigned": {robot.robot_id for robot in robots if robot.assigned},
        "unassigned": set(unassigned_robots)
    }
    time_steps_unchanged = 0
    
    for time_step in range(max_time_steps):
        # print(f"\n--- Time Step {time_step + 1} ---")
        # print(f"OCCUPIED POSITIONS: {occupied_positions}")
        # print(f"START POSITIONS: {start_positions}")
        # print(f"GOAL POSITONS: {goal_positions}")
        # print(f"UNASSIGNED ROBOTS: {unassigned_robots}")
        # print(f"UNASSIGNED TASKS: {unassigned_tasks}")
        # # print(f"ASSIGNED ROBOTS: {[rob.assigned for rob in robots]}")
        # print(f"ALL ROBOTS: {len(robots)}")
        # print(f"ALL TASKS: {len(tasks)}")
        # print(f"LIST OF ALL ROBOTS: {[rob.robot_id for rob in robots]}")
        # print(f"LIST OF ALL TASKS: {[tas.task_id for tas in tasks]}")

        unassigned_robots = [r.robot_id for r in robots if not r.assigned]
        unassigned_tasks = [t.task_id for t in tasks if not t.assigned]

        unassigned_count, total_reward, total_success = simulate_time_step(robots, tasks, unassigned_robots, unassigned_tasks, suitability_method, occupied_positions, start_positions, goal_positions, 1.0, total_reward, total_success)
        if add_tasks and time_step + 1 <= 2 and random.random() < 0.5: # add tasks only in the first 2 time steps, and randomly
            print(f"ADDING NEW TASKS AT TIME STEP {time_step + 1}")
            task_max_id, total_tasks = add_new_tasks(tasks, unassigned_tasks, task_max_id, random.randint(0, tasks_to_add), total_tasks, grid, occupied_positions)
        if add_robots and time_step + 1 <= 4 and random.random() < 0.5: # add robots only in the first 4 time steps, and randomly
            print(f"ADDING NEW ROBOTS AT TIME STEP {time_step + 1}")
            robot_max_id = add_new_robots(robots, unassigned_robots, robot_max_id, random.randint(0, robots_to_add), grid, occupied_positions)

        if remove_robots and time_step + 1 <= 4 and random.random() < 0.5: # remove robots only in the first 4 time steps, and randomly
            if len(assigned_robots) > 1:
                print(f"REMOVING RANDOM ROBOTS AT TIME STEP {time_step + 1}")
                remove_random_robots(robots, tasks, unassigned_robots, unassigned_tasks, random.randint(0, robots_to_remove), occupied_positions, start_positions, goal_positions)

        if unassigned_robots and unassigned_tasks:
            total_reassignments += 1
            _, unassigned_robots, unassigned_tasks, reassign_score, reassign_length = reassign_robots_to_tasks_with_method(
                robots, tasks, num_candidates, voting_method, suitability_method, 
                unassigned_robots, unassigned_tasks, ssia_task_allocation, start_positions, goal_positions)
            total_reassignment_time += reassign_length
            total_reassignment_score += reassign_score

            # regenerate the master assigned robots dictionary based on flags
            assigned_robots = {
                r.robot_id: r.current_task.task_id 
                for r in robots 
                if r.assigned and r.current_task is not None
            }
            print(f"ROBOTS REASSIGNED")
            print(f"ASSIGNED ROBOTS: {assigned_robots}")
            print(f"ALL ROBOTS: {[robot.robot_id for robot in robots]}")

        for robot in robots:
            if robot.assigned and robot.current_task is not None:
                start_positions[robot.robot_id] = robot.location
                goal_positions[robot.robot_id] = robot.current_task.location
            else:
                start_positions.pop(robot.robot_id, None)
                goal_positions.pop(robot.robot_id, None)
        
        current_state = {
            "robot_ids": set(robot.robot_id for robot in robots),
            "assigned": {robot.robot_id for robot in robots if robot.assigned},
            "unassigned": set(unassigned_robots)
        }

        if current_state != prev_state and [rob.assigned for rob in robots].count(True) >= 1:
            print("State change, re-run CBS...")
            print(f"AMOUNT OF ASSIGNED ROBOTS: {len([rob.current_task for rob in robots])}")
            print(f"ASSIGNED ROBOTS: {[rob.assigned for rob in robots].count(True)}")
            print(f"OCCUPIED POSITIONS: {occupied_positions}")
            print(f"START POSITIONS: {start_positions}")
            print(f"GOAL POSITONS: {goal_positions}")
            print(f"UNASSIGNED ROBOTS: {unassigned_robots}")
            print(f"UNASSIGNED TASKS: {unassigned_tasks}")
            print(f"ALL ROBOTS: {len(robots)}")
            print(f"ALL TASKS: {len(tasks)}")
            print(f"LIST OF ALL ROBOTS: {[rob.robot_id for rob in robots]}")
            print(f"LIST OF ALL TASKS: {[tas.task_id for tas in tasks]}")
            print(f"SUITABILITY MATRIX AT TIMESTEP {time_step}: {suitability_matrix}")

            agents = build_cbs_agents(robots, start_positions, goal_positions)

            # Validation block
            start_locations = [agent['start'] for agent in agents]
            if len(start_locations) != len(set(start_locations)):
                print("ERROR: Duplicate start locations found in agent list. Aborting CBS.")
                # You can decide how to handle this. For now, we can skip the planning step for this cycle.
                # To prevent a crash, you could set solution to None and handle it.
                solution = None
            else:
                # instantiate environment and planner
                env = Environment(
                    dimension=map_dict['dimension'],
                    agents=agents,
                    obstacles=map_dict['obstacles']
                )
                planner = CBS(env)
                solution, nodes_expanded, conflicts = planner.search()
                print(f"CBS COMPLETE. New solution: {solution}")

            if solution:
                # update each robot's path based on the new CBS solution
                for robot_id, schedule in solution.items():
                    ridx = [robot.robot_id for robot in robots].index(robot_id)
                    for point in schedule:
                        # print(f"  t: {point['t']}, x: {point['x']}, y: {point['y']}")
                        # Assign the paths to the robots
                        robots[ridx].current_path.append((point['x'], point['y']))
                    # ridx = [r.robot_id for r in robots].index(robot_id)
                    # robots[ridx].current_path = schedule
                    robots[ridx].remaining_distance = len(schedule) - 1
                prev_state = current_state.copy()

            else:
                print("CBS could not find a conflict free path assignment for all agents")
                # could possibly fall back on the simple method here if we get a lot of issues
                # but for now, we will just skip CBS and continue with the simulation
                print("Skipping CBS...")
                time_steps_unchanged += 1
                if time_steps_unchanged >= 3:
                    print("No state change for 3 time steps, ending simulation.")
                    break
        else:
            print("No state change, skip CBS...")
            print(f"ALL ROBOTS: {len(robots)}")
            print(f"ALL TASKS: {len(tasks)}")
            print(f"LIST OF ALL ROBOTS: {[rob.robot_id for rob in robots]}")
            print(f"LIST OF ALL TASKS: {[tas.task_id for tas in tasks]}")

    overall_success_rate = total_success / total_tasks
    print(f"SSIA: Total reward: {total_reward}, Overall success rate: {overall_success_rate:.2%}, Tasks completed: {total_success}, Reassignment Time: {total_reassignment_time}, Reassignment Score: {total_reassignment_score}, total reassignments: {total_reassignments}")
# #     for robot in robots:
# #         print(f"Robot {robot.robot_id} attempted {robot.tasks_attempted} tasks and successfully completed {robot.tasks_successful} of them.")

    robots = copy.deepcopy(initial_robots)
    tasks = copy.deepcopy(initial_tasks)
    robot_max_id = len(robots)+1
    task_max_id = len(tasks)+1
    total_reward = 0.0
    total_success = 0.0
    total_tasks = num_tasks
    total_reassignment_time = 0.0
    total_reassignment_score = 0.0
    total_reassignments = 0

    # propogate the initial occupied positions again
    initial_positions = set()
    # for r in robots:
    #     row = int(r.location[0][0])
    #     col = int(r.location[0][1])
    #     initial_positions.add((row, col))
    for r in robots:
        # add the robot's initial position to the occupied positions set
        initial_positions.add(r.location)

    occupied_positions = set(initial_positions)

    assigned_pairs = ilp_output[0]
    for robot_idx, task_idx in assigned_pairs:
        r = robots[robot_idx]
        t = tasks[task_idx]

        r.current_task = t
        r.assigned = True
        r.tasks_attempted = 1

        t.assigned_robot = r
        t.assigned = True
    
    # rebuild all assignment dictionaries from the robot and task flags
    assigned_robots = {r.robot_id: r.current_task.task_id for r in
                            robots if r.assigned and r.current_task is not None}
    unassigned_tasks = [t.task_id for t in tasks if not t.assigned]
    unassigned_robots = [r.robot_id for r in robots if not r.assigned]

    start_positions = {
        r.robot_id: r.location
        for r in robots
        if r.assigned and r.current_task is not None
    }

    goal_positions = {
        r.robot_id: r.current_task.location
        for r in robots
        if r.assigned and r.current_task is not None
    }

    print(f"ROBOTS: {[rob.robot_id for rob in robots]}")
    print(f"TASKS: {[tas.task_id for tas in tasks]}")
    print(f"ASSIGNED ROBOTS: {assigned_robots}")

    agents = build_cbs_agents(robots, start_positions, goal_positions)

    input_data = {
        'map' : {
            'dimension': map_dict['dimension'],
            'obstacles': map_dict['obstacles']
        },
        'agents': agents,
    }

    env = Environment(
        dimension=map_dict['dimension'],
        agents=input_data['agents'],
        obstacles=input_data['map']['obstacles'],
    )
    planner = CBS(env)
    solution, nodes_expanded, conflicts = planner.search()

    print(f"SOLUTION: {solution}")

    if solution is None:
        print("CBS could not find a conflict free path assignment for all agents")

    else:
        for agent, schedule in solution.items():
            print(f"Schedule for {agent}:")

            for point in schedule:
                rid = [robot.robot_id for robot in robots].index(agent)
                robots[rid].current_path.append((point['x'], point['y']))
            print(f"Robot {agent} path: {robots[rid].current_path}")

    prev_state = {
        "robot_ids": set(robot.robot_id for robot in robots),
        "assigned": {robot.robot_id for robot in robots if robot.assigned},
        "unassigned": set(unassigned_robots)
    }
    time_steps_unchanged = 0

    for time_step in range(max_time_steps):

        unassigned_robots = [r.robot_id for r in robots if not r.assigned]
        unassigned_tasks = [t.task_id for t in tasks if not t.assigned]

        unassigned_count, total_reward, total_success = simulate_time_step(robots, tasks, unassigned_robots, unassigned_tasks, suitability_method, occupied_positions, start_positions, goal_positions, 1.0, total_reward, total_success)
        if add_tasks and time_step + 1 <= 2 and random.random() < 0.5: # add tasks only in the first 2 time steps, and randomly
            print(f"ADDING NEW TASKS AT TIME STEP {time_step + 1}")
            task_max_id, total_tasks = add_new_tasks(tasks, unassigned_tasks, task_max_id, random.randint(0, tasks_to_add), total_tasks, grid, occupied_positions)
        if add_robots and time_step + 1 <= 4 and random.random() < 0.5: # add robots only in the first 4 time steps, and randomly
            print(f"ADDING NEW ROBOTS AT TIME STEP {time_step + 1}")
            robot_max_id = add_new_robots(robots, unassigned_robots, robot_max_id, random.randint(0, robots_to_add), grid, occupied_positions)
        
        if remove_robots and time_step + 1 <= 4 and random.random() < 0.5: # remove robots only in the first 4 time steps, and randomly
            if len(assigned_robots) > 1:
                print(f"REMOVING RANDOM ROBOTS AT TIME STEP {time_step + 1}")
                remove_random_robots(robots, tasks, unassigned_robots, unassigned_tasks, random.randint(0, robots_to_remove), occupied_positions, start_positions, goal_positions)

        if unassigned_robots and unassigned_tasks:
            total_reassignments += 1
            _, unassigned_robots, unassigned_tasks, reassign_score, reassign_length = reassign_robots_to_tasks_with_method(
                robots, tasks, num_candidates, voting_method, suitability_method, 
                unassigned_robots, unassigned_tasks, ilp_task_allocation, start_positions, goal_positions)
            total_reassignment_time += reassign_length
            total_reassignment_score += reassign_score

            # regenerate the master assigned robots dictionary based on flags
            assigned_robots = {
                r.robot_id: r.current_task.task_id 
                for r in robots 
                if r.assigned and r.current_task is not None
            }
            print(f"ROBOTS REASSIGNED")
            print(f"ASSIGNED ROBOTS: {assigned_robots}")
            print(f"ALL ROBOTS: {[robot.robot_id for robot in robots]}")

        for robot in robots:
            if robot.assigned and robot.current_task is not None:
                start_positions[robot.robot_id] = robot.location
                goal_positions[robot.robot_id] = robot.current_task.location
            else:
                start_positions.pop(robot.robot_id, None)
                goal_positions.pop(robot.robot_id, None)

        current_state = {
            "robot_ids": set(robot.robot_id for robot in robots),
            "assigned": {robot.robot_id for robot in robots if robot.assigned},
            "unassigned": set(unassigned_robots)
        }

        if current_state != prev_state and [rob.assigned for rob in robots].count(True) >= 1:
            print("State change, re-run CBS...")
            print(f"AMOUNT OF ASSIGNED ROBOTS: {len([rob.current_task for rob in robots])}")
            print(f"ASSIGNED ROBOTS: {[rob.assigned for rob in robots].count(True)}")
            print(f"OCCUPIED POSITIONS: {occupied_positions}")
            print(f"START POSITIONS: {start_positions}")
            print(f"GOAL POSITONS: {goal_positions}")
            print(f"UNASSIGNED ROBOTS: {unassigned_robots}")
            print(f"UNASSIGNED TASKS: {unassigned_tasks}")
            print(f"ALL ROBOTS: {len(robots)}")
            print(f"ALL TASKS: {len(tasks)}")
            print(f"LIST OF ALL ROBOTS: {[rob.robot_id for rob in robots]}")
            print(f"LIST OF ALL TASKS: {[tas.task_id for tas in tasks]}")
            print(f"SUITABILITY MATRIX AT TIMESTEP {time_step}: {suitability_matrix}")

            agents = build_cbs_agents(robots, start_positions, goal_positions)

            # Validation block
            start_locations = [agent['start'] for agent in agents]
            if len(start_locations) != len(set(start_locations)):
                print("ERROR: Duplicate start locations found in agent list. Aborting CBS.")
                # You can decide how to handle this. For now, we can skip the planning step for this cycle.
                # To prevent a crash, you could set solution to None and handle it.
                solution = None
            else:
                # instantiate environment and planner
                env = Environment(
                    dimension=map_dict['dimension'],
                    agents=agents,
                    obstacles=map_dict['obstacles']
                )
                planner = CBS(env)
                solution, nodes_expanded, conflicts = planner.search()
                print(f"CBS COMPLETE. New solution: {solution}")

            if solution:
                # update each robot's path based on the new CBS solution
                for robot_id, schedule in solution.items():
                    ridx = [robot.robot_id for robot in robots].index(robot_id)
                    for point in schedule:
                        # print(f"  t: {point['t']}, x: {point['x']}, y: {point['y']}")
                        # Assign the paths to the robots
                        robots[ridx].current_path.append((point['x'], point['y']))
                    # ridx = [r.robot_id for r in robots].index(robot_id)
                    # robots[ridx].current_path = schedule
                    robots[ridx].remaining_distance = len(schedule) - 1
                prev_state = current_state.copy()

            else:
                print("CBS could not find a conflict free path assignment for all agents")
                # could possibly fall back on the simple method here if we get a lot of issues
                # but for now, we will just skip CBS and continue with the simulation
                print("Skipping CBS...")
                time_steps_unchanged += 1
                if time_steps_unchanged >= 3:
                    print("No state change for 3 time steps, ending simulation.")
                    break
        else:
            print("No state change, skip CBS...")
            print(f"ALL ROBOTS: {len(robots)}")
            print(f"ALL TASKS: {len(tasks)}")
            print(f"LIST OF ALL ROBOTS: {[rob.robot_id for rob in robots]}")
            print(f"LIST OF ALL TASKS: {[tas.task_id for tas in tasks]}")

    overall_success_rate = total_success / total_tasks
    print(f"ILP: Total reward: {total_reward}, Overall success rate: {overall_success_rate:.2%}, Tasks completed: {total_success}, Reassignment Time: {total_reassignment_time}, Reassignment Score: {total_reassignment_score}")
# #     for robot in robots:
# #         print(f"Robot {robot.robot_id} attempted {robot.tasks_attempted} tasks and successfully completed {robot.tasks_successful} of them.")

def benchmark_simulation(num_robots: int, num_tasks: int, num_candidates: int, voting_method: str, suitability_method: str, max_time_steps: int, add_tasks: bool, add_robots: bool, remove_robots: bool, tasks_to_add: int = 1, robots_to_add: int = 1, robots_to_remove: int = 1):
    start_time = time.time()
    main_simulation(num_robots, num_tasks, num_candidates, voting_method, suitability_method, max_time_steps, add_tasks, add_robots, remove_robots, tasks_to_add, robots_to_add, robots_to_remove)
    end_time = time.time()
    execution_time = end_time - start_time

    cpu_usage = psutil.cpu_percent()
    memory_usage = psutil.virtual_memory().used

    print(f"Simulation completed in {execution_time:.2f} seconds.")
    print(f"CPU Usage: {cpu_usage}%")
    print(f"Memory Usage: {memory_usage / (1024 * 1024)} MB")

if __name__ == "__main__":
#     voting_methods = ["rank_assignments_borda", "rank_assignments_approval", "rank_assignments_majority_judgment", "rank_assignments_cumulative_voting", "rank_assignments_condorcet_method", "rank_assignments_range"]
    voting_methods = ["rank_assignments_range"]
#     suitability_methods = ["evaluate_suitability_loose", "evaluate_suitability_strict", "evaluate_suitability_distance", "evaluate_suitability_priority"]
    # suitability_methods = ["evaluate_suitability_loose","evaluate_suitability_distance"]
    suitability_methods = ["evaluate_suitability_loose"]
    # suitability_methods = ["evaluate_suitability_distance"]
    max_time_steps = 100
    add_tasks = True
    add_robots = False
    remove_robots = False
    # param_combinations = []
    for i in [5]:
        for j in [5]:
            for nc in [5]:
                for vm in voting_methods:
                    for sm in suitability_methods:
                        for k in range(0,10):
                            # param_combinations.append((i, j, nc, vm, sm, max_time_steps, add_tasks, add_robots, remove_robots, 10, 10, 10))
    # with multiprocessing.Pool() as pool:
        # pool.starmap(main_simulation, param_combinations)
                            # main_simulation(i, j, nc, vm, sm, max_time_steps, add_tasks, add_robots, remove_robots,10,10,10)
                            benchmark_simulation(i, j, nc, vm, sm, max_time_steps, add_tasks, add_robots, remove_robots,10,10,10)

SUITABILITY METHOD: evaluate_suitability_loose
Score of the Best Assignment: 65.89
Time taken for the voting process: 8.70 microseconds
ROBOTS: ['R1', 'R2', 'R3', 'R4', 'R5']
TASKS: ['T1', 'T2', 'T3', 'T4', 'T5']
ASSIGNED ROBOTS: {'R1': 'T3', 'R2': 'T4', 'R3': 'T2', 'R4': 'T5', 'R5': 'T1'}
Total Suitability Score: 67.39
Time taken for allocation: 33.50 microseconds
Suitability Matrix Issue: [[10.95739675 13.33122673 11.87342967 16.82728107 11.39683806]
 [ 8.95739675 13.83122673  8.87342967 14.82728107  8.39683806]
 [ 8.45739675 16.33122673  9.37342967 15.32728107  9.89683806]
 [ 9.45739675 17.83122673 10.37342967 17.32728107 10.89683806]
 [11.95739675 14.33122673 11.87342967 18.82728107 12.39683806]]
Number of Robots: 5, Number of Tasks: 5
Robot 0 bids 10.957396749321422 for Task 0
Robot 4 bids 11.957396749321422 for Task 0
Robot 0 bids 13.331226725891323 for Task 1
Robot 1 bids 13.831226725891323 for Task 1
Robot 2 bids 16.331226725891323 for Task 1
Robot 3 bids 17.831226725891323 for

ValueError: not enough values to unpack (expected 3, got 0)

In [17]:
from transformers import AutoModelForCausalLM, AutoTokenizer

checkpoint = "bigscience/bloomz-560m"

# tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# model = AutoModelForCausalLM.from_pretrained(checkpoint)

# inputs = tokenizer.encode(prompt[0], return_tensors="pt")
# outputs = model.generate(inputs, max_new_tokens=600)
# print(tokenizer.decode(outputs[0]))

In [18]:
import huggingface_hub
huggingface_hub.login('***REMOVED***')

In [19]:
model_name = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.98s/it]


In [20]:
model = model.to("cuda")

# Function to generate text
def generate_text(prompt, max_length=1000):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(inputs.input_ids, max_length=max_length, do_sample=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test the model
prompt = "Once upon a time in a land far, far away,"
print(generate_text(prompt))


Once upon a time in a land far, far away, there was a magical kingdom where all the creatures lived in harmony. But one day, a dark force threatened to destroy their peaceful way of life. The creatures banded together to defend their home, and in the process, they discovered the true meaning of courage and friendship.

In this kingdom, there lived a wise old owl named Hootie who was known for his wisdom and insight. Hootie had been watching the creatures of the kingdom for many years and had seen the dark force growing stronger by the day. He knew that something had to be done to stop it, but he didn't know what.

One day, Hootie called a meeting of all the creatures of the kingdom to discuss the problem. The creatures gathered around him, their eyes filled with fear and uncertainty. Hootie looked at them and said, "My dear friends, we are in grave danger. The dark force that threatens our kingdom is growing stronger every day. We must find a way to stop it before it's too late."

The 

In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import MultiComparison
from statsmodels.stats.anova import anova_lm

# Load the data
file_path = 'VBTAData.csv'  # Replace with your file path
data = pd.read_csv(file_path)

# Define categorical and continuous variables
categorical_factors = ['Voting_Method', 'Suitability_Method']  # Replace with actual categorical column names
continuous_factors = ['Robots', 'Tasks']   # Replace with actual continuous column names
# dependent_var = 'Score_Voting'                   # Replace with your dependent variable name
dependent_var = 'Time_Voting'                   # Replace with your dependent variable name

# Convert categorical variables to category dtype
for factor in categorical_factors:
    data[factor] = data[factor].astype('category')

# Define the formula for the ANCOVA model
formula = f"{dependent_var} ~ {' * '.join(categorical_factors)} + {' + '.join(continuous_factors)}"

# Fit the ANCOVA model
model = ols(formula, data=data).fit()
anova_table = sm.stats.anova_lm(model, typ=2)  # Type 2 ANOVA/ANCOVA

# Print ANCOVA results
print("Two-way ANCOVA results:")
print(anova_table)

# Post hoc analysis: Pairwise comparisons on adjusted means
print("\nPost hoc analysis (pairwise comparisons):")
for factor in categorical_factors:
    print(f"\nComparisons within {factor} (adjusted for covariates):")
    
    # Pairwise comparisons using t-tests
    mc = MultiComparison(data[dependent_var], data[factor])
    tukey_result = mc.tukeyhsd()
    
    # Display Tukey's test results for adjusted means
    print(tukey_result)

Two-way ANCOVA results:
                                        sum_sq       df            F  \
Voting_Method                     3.174761e+15      4.0   344.271446   
Suitability_Method                3.633505e+12      3.0     0.525357   
Voting_Method:Suitability_Method  5.143658e+12     12.0     0.185926   
Robots                            1.054736e+16      1.0  4575.028956   
Tasks                             1.140436e+16      1.0  4946.761087   
Residual                          4.513551e+16  19578.0          NaN   

                                         PR(>F)  
Voting_Method                     6.711993e-287  
Suitability_Method                 6.648329e-01  
Voting_Method:Suitability_Method   9.989597e-01  
Robots                             0.000000e+00  
Tasks                              0.000000e+00  
Residual                                    NaN  

Post hoc analysis (pairwise comparisons):

Comparisons within Voting_Method (adjusted for covariates):
                 

In [22]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Load the data
file_path = 'VBTAData.csv'  # Replace with your file path
data = pd.read_csv(file_path)

# Define column names
# group_columns = ['Score_Voting', 'Score_CBBA', 'Score_SSIA', 'Score_ILP']  # Replace with your actual column names for each group
group_columns = ['Time_Voting', 'Time_CBBA', 'Time_SSIA', 'Time_ILP']  # Replace with your actual column names for each group
covariates = ['Robots', 'Tasks']  # Replace with actual covariate column names

data = data[data['Voting_Method'] == 'rank_assignments_range']

data = data[data['Suitability_Method'] == 'evaluate_suitability_loose']

# Melt the data from wide to long format
data_long = data.melt(id_vars=covariates, value_vars=group_columns, var_name='group', value_name='dependent')

# Convert the group variable to categorical
data_long['group'] = data_long['group'].astype('category')

# Define the formula for the ANCOVA model
formula = f"dependent ~ group + {' + '.join(covariates)}"

# Fit the ANCOVA model
model = ols(formula, data=data_long).fit()
anova_table = sm.stats.anova_lm(model, typ=2)  # Type 2 ANCOVA for better control over interactions

# Print ANCOVA results
print("ANCOVA results:")
print(anova_table)

# Check if the group factor is significant
if anova_table.loc['group', 'PR(>F)'] < 0.05:
    print("\nThe group factor is significant. Proceeding with post hoc tests.")
    
    # Post hoc analysis using Tukey's HSD
    print("\nPost hoc analysis (Tukey's HSD on adjusted means):")
    tukey_result = pairwise_tukeyhsd(data_long['dependent'], data_long['group'], alpha=0.05)
    print(tukey_result)
else:
    print("\nThe group factor is not significant. No post hoc test needed.")


ANCOVA results:
                sum_sq      df           F         PR(>F)
group     5.865748e+14     3.0  233.766498  1.620202e-139
Robots    7.722562e+14     1.0  923.297264  2.858098e-182
Tasks     5.819917e+14     1.0  695.820039  2.786051e-141
Residual  3.273714e+15  3914.0         NaN            NaN

The group factor is significant. Proceeding with post hoc tests.

Post hoc analysis (Tukey's HSD on adjusted means):
            Multiple Comparison of Means - Tukey HSD, FWER=0.05            
  group1     group2     meandiff   p-adj      lower        upper     reject
---------------------------------------------------------------------------
Time_CBBA    Time_ILP  476152.4754    0.0   349931.5305  602373.4203   True
Time_CBBA   Time_SSIA  -477618.259    0.0  -603839.2039 -351397.3141   True
Time_CBBA Time_Voting -438219.1265    0.0  -564440.0714 -311998.1816   True
 Time_ILP   Time_SSIA -953770.7344    0.0 -1079991.6793 -827549.7895   True
 Time_ILP Time_Voting -914371.6019    0.0 -1